In [ ]:
from collections import Counter

def get_deck_counts(game):
    """Return a dict of rank -> count for cards that could still be in the deck (unknown cards)."""
    # Start with a full deck (4 of each rank)
    full_deck_counts = {rank: 4 for rank in ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']}

    # Subtract all known public cards
    for player in game.players:
        for i, card in enumerate(player.grid):
            if card and player.known[i]:  # Card exists and is public
                full_deck_counts[card.rank] -= 1

    # Subtract cards in the discard pile (they are public)
    if game.discard_pile:
        for card in game.discard_pile:
            full_deck_counts[card.rank] -= 1

    # Ensure no negative counts
    for rank in full_deck_counts:
        full_deck_counts[rank] = max(0, full_deck_counts[rank])

    return full_deck_counts


def get_private_deck_counts(game):
    """Return a dict of rank -> count for full_deck count - the players private cards."""
    # Start with a full deck (4 of each rank)
    full_deck_counts = {rank: 4 for rank in ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']}

    # Subtract all known public cards
    for player in game.players:
        for i, card in enumerate(player.grid):
            if card and player.known[i]:  # Card exists and is public
                full_deck_counts[card.rank] -= 1

    # Subtract cards in the discard pile (they are public)
    if game.discard_pile:
        for card in game.discard_pile:
            full_deck_counts[card.rank] -= 1

    # Subtract human player's private cards (bottom 2 face-down cards they saw initially)
    human_player = game.players[0]  # Human is always player 0
    for i, card in enumerate(human_player.grid):
        if card and not human_player.known[i] and i >= 2:  # Bottom 2 cards (positions 2,3) that are not public
            full_deck_counts[card.rank] -= 1

    # Ensure no negative counts
    for rank in full_deck_counts:
        full_deck_counts[rank] = max(0, full_deck_counts[rank])

    return full_deck_counts

def prob_draw_lower_than_min_faceup(game, player=None):
    """For a specific player, probability that next card is lower than their lowest visible card."""
    deck = game.deck
    if not deck:
        return '0.0%'

    # Use specified player or default to first player for backwards compatibility
    target_player = player if player is not None else game.players[0]

    # For human player, include both known (public) and privately_visible cards
    # For AI players, only include known (public) cards
    if target_player.agent_type == 'human':
        # Include cards that are either publicly known or privately visible
        visible_cards = [card for i, card in enumerate(target_player.grid)
                       if card and (target_player.known[i] or target_player.privately_visible[i])]
    else:
        # For AI players, only include publicly known cards
        visible_cards = [card for i, card in enumerate(target_player.grid)
                       if card and target_player.known[i]]

    if not visible_cards:
        return '0.0%'

    min_val = min(card.score() for card in visible_cards)
    lower = [card for card in deck if card.score() < min_val]
    prob = len(lower) / len(deck)
    return f'{round(prob * 100, 1)}%'




def prob_draw_pair(game, player=None):
    """For a specific player, probability that next card matches any rank in their visible grid."""
    deck = game.deck
    if not deck:
        return '0.0%'

    # Use specified player or default to first player for backwards compatibility
    target_player = player if player is not None else game.players[0]

    # For human player, include both known (public) and privately_visible cards
    # For AI players, only include known (public) cards
    if target_player.agent_type == 'human':
        # Include cards that are either publicly known or privately visible
        visible_cards = [card for i, card in enumerate(target_player.grid)
                       if card and (target_player.known[i] or target_player.privately_visible[i])]
    else:
        # For AI players, only include publicly known cards
        visible_cards = [card for i, card in enumerate(target_player.grid)
                       if card and target_player.known[i]]

    ranks_in_grid = set(card.rank for card in visible_cards)
    matching = [card for card in deck if card.rank in ranks_in_grid]
    prob = len(matching) / len(deck)
    return f'{round(prob * 100, 1)}%'


def prob_improve_hand(game, player=None):
    """
    For a specific player, return the probability that drawing the next card would improve their hand,
    either by:
    - Forming a pair with any card in their visible grid, or
    - Being lower than any card in their visible grid (for a potential swap).
    """
    deck = game.deck
    if not deck:
        return '0.0%'

    # Use specified player or default to first player for backwards compatibility
    target_player = player if player is not None else game.players[0]

    # For human player, include both known (public) and privately_visible cards
    # For AI players, only include known (public) cards
    if target_player.agent_type == 'human':
        # Include cards that are either publicly known or privately visible
        visible_cards = [card for i, card in enumerate(target_player.grid)
                       if card and (target_player.known[i] or target_player.privately_visible[i])]
    else:
        # For AI players, only include publicly known cards
        visible_cards = [card for i, card in enumerate(target_player.grid)
                       if card and target_player.known[i]]

    if not visible_cards:
        return '0.0%'

    all_ranks = set(card.rank for card in visible_cards)
    all_scores = [card.score() for card in visible_cards]

    improving_cards = 0
    for card in deck:
        makes_pair = card.rank in all_ranks
        beats_known = any(card.score() < s for s in all_scores)
        if makes_pair or beats_known:
            improving_cards += 1

    prob = improving_cards / len(deck)
    return f'{round(prob * 100, 1)}%'



def get_probabilities(game):
    """Return a dict of interesting probabilities/statistics for the current game state."""
    # Calculate probabilities for each player to maintain backwards compatibility
    prob_draw_lower_results = []
    prob_draw_pair_results = []
    prob_improve_hand_results = []

    for player in game.players:
        prob_draw_lower_results.append(prob_draw_lower_than_min_faceup(game, player))
        prob_draw_pair_results.append(prob_draw_pair(game, player))
        prob_improve_hand_results.append(prob_improve_hand(game, player))

    return {
        'deck_counts': get_deck_counts(game),
        'private_deck_counts': get_private_deck_counts(game),
        'prob_draw_lower_than_min_faceup': prob_draw_lower_results,
        'prob_draw_pair': prob_draw_pair_results,
        'prob_improve_hand': prob_improve_hand_results,
        'expected_value_draw_vs_discard': expected_value_draw_vs_discard(game),
        'average_deck_score': round(average_score_of_deck(game), 2) if game.deck else 0,
    }

def expected_score_blind(grid, known, rank_probabilities, privately_visible=None):
    """
    Compute the expected score of a grid, using:
      - True values for known cards
      - Probability-weighted expected values for unknown cards
      - For pairs: count a pair if both cards are known or (for human) privately visible
    """
    from models import Card
    scores = []
    ranks = []
    for i in range(4):
        if known[i] and grid[i]:
            scores.append(grid[i].score())
            ranks.append(grid[i].rank)
        elif privately_visible is not None and privately_visible[i] and grid[i]:
            scores.append(grid[i].score())
            ranks.append(grid[i].rank)
        else:
            # Use expected value for unknown
            expected = sum(Card(rank, '♠').score() * prob for rank, prob in rank_probabilities.items())
            scores.append(expected)
            # For pairing, treat as unknown (None)
            ranks.append(None)
    total_score = sum(scores)
    # Only count pairs if both cards are known or privately visible
    used = set()
    for pos1 in range(4):
        for pos2 in range(pos1+1, 4):
            if (ranks[pos1] is not None and ranks[pos2] is not None and
                ranks[pos1] == ranks[pos2] and pos1 not in used and pos2 not in used):
                # For human, check privately_visible as well
                if (known[pos1] or (privately_visible is not None and privately_visible[pos1])) and \
                   (known[pos2] or (privately_visible is not None and privately_visible[pos2])):
                    # Subtract both scores (they become zero)
                    total_score -= (scores[pos1] + scores[pos2])
                    used.add(pos1)
                    used.add(pos2)
    return total_score

def expected_value_draw_vs_discard(game, player=None):
    """
    Calculate the expected value (EV) of drawing from the deck vs taking the discard card for the specified player.
    If no player is specified, defaults to game.players[0] (for backwards compatibility).

    In Golf, **lower scores are better**. Here, EV is defined as the **expected change in score**:
        - A **negative EV** means your score is expected to go down (good).
        - A **positive EV** means your score is expected to go up (bad).

    The function returns a dict with:
        - draw_expected_value: Expected change in score if you draw from the deck (averaged over all possible draws)
        - discard_expected_value: Best possible change in score if you take the discard card
        - recommendation: Which action is better (draw or discard)
        - draw_advantage: Difference between draw_expected_value and discard_expected_value (negative = draw is better)

    Calculation details:
    1. **Discard EV**: For each available position in your grid, try swapping in the discard card and calculate the change (test_score - current_score). The best (most negative) change is used as the discard EV.
    2. **Draw EV**: For each possible card you could draw, calculate the best (most negative) change (swap or flip), weighted by probability
    3. **Interpretation**: Negative EV means a drop in your score (good in golf).
    """
    if not game.deck or not game.discard_pile:
        return {
            'draw_expected_value': 0,
            'discard_expected_value': 0,
            'recommendation': 'No valid comparison possible',
            'draw_advantage': 0
        }

    # Use specified player or default to first player for backwards compatibility
    target_player = player if player is not None else game.players[0]
    discard_card = game.discard_pile[-1]

    # Get available positions for target player (face-down cards)
    available_positions = [i for i in range(4) if not target_player.known[i]]

    if not available_positions:
        return {
            'draw_expected_value': 0,
            'discard_expected_value': 0,
            'recommendation': 'No available positions',
            'draw_advantage': 0
        }

    # Get probabilities for unknown cards
    private_deck_counts = get_private_deck_counts(game)
    total_private = sum(private_deck_counts.values())
    rank_probabilities = {rank: count / total_private if total_private > 0 else 0 for rank, count in private_deck_counts.items()}

    # Calculate current hand score using expected_score_blind
    current_score = expected_score_blind(target_player.grid, target_player.known, rank_probabilities, getattr(target_player, 'privately_visible', None))

    # --- Discard EV ---
    # Try placing discard card in each available position and find best (most negative) change
    best_discard_ev = 0
    best_discard_position = None
    from models import Card

    for pos in available_positions:
        # For human, treat privately_visible as known
        is_known = target_player.known[pos]
        is_private = hasattr(target_player, 'privately_visible') and target_player.privately_visible[pos]
        if (is_known or is_private) and target_player.grid[pos]:
            current_card_score = target_player.grid[pos].score()
        else:
            # If unknown, use expected score
            current_card_score = sum(Card(rank, '♠').score() * prob for rank, prob in rank_probabilities.items())
        # Simulate swapping in the discard card
        test_grid = target_player.grid.copy()
        test_grid[pos] = discard_card
        test_known = target_player.known.copy()
        test_known[pos] = True  # After swap, this card is known
        # For human, update privately_visible as well
        test_privately_visible = getattr(target_player, 'privately_visible', None)
        if test_privately_visible is not None:
            test_privately_visible = test_privately_visible.copy()
            test_privately_visible[pos] = True
        test_score = expected_score_blind(test_grid, test_known, rank_probabilities, test_privately_visible)
        ev = test_score - current_score
        if ev < best_discard_ev or best_discard_position is None:
            best_discard_ev = ev
            best_discard_position = pos

    discard_expected_value = best_discard_ev

    # --- Draw EV ---
    # For each possible card you could draw, calculate the best (most negative) change (swap or flip), weighted by probability
    deck_counts = get_private_deck_counts(game)
    total_remaining_cards = sum(deck_counts.values())

    if total_remaining_cards == 0:
        draw_expected_value = 0
        best_draw_position = None
        best_flip_position = None
        best_action_type = "keep"  # "keep" or "flip"
    else:
        draw_expected_value = 0
        best_overall_ev = float('inf')
        best_draw_position = None
        best_flip_position = None
        best_action_type = "keep"

        for rank, count in deck_counts.items():
            if count > 0:
                drawn_card = Card(rank, '♠')  # Suit doesn't matter for score

                # Step 1: Evaluate keeping the drawn card (swap into each available position)
                best_draw_ev = float('inf')
                current_best_draw_position = None
                for pos in available_positions:
                    # If the card is known, use its actual value
                    if target_player.known[pos] and target_player.grid[pos]:
                        current_card_score = target_player.grid[pos].score()
                    else:
                        # If unknown, use expected score
                        current_card_score = sum(Card(r, '♠').score() * prob for r, prob in rank_probabilities.items())
                    test_grid = target_player.grid.copy()
                    test_grid[pos] = drawn_card
                    test_known = target_player.known.copy()
                    test_known[pos] = True  # After swap, this card is known
                    test_score = expected_score_blind(test_grid, test_known, rank_probabilities, getattr(target_player, 'privately_visible', None))
                    ev = test_score - current_score
                    if ev < best_draw_ev:
                        best_draw_ev = ev
                        current_best_draw_position = pos

                # Step 2: Evaluate discarding the drawn card and flipping one of your own
                # Calculate actual expected score change when flipping each position
                best_flip_ev = float('inf')
                current_best_flip_position = None
                for flip_pos in available_positions:
                    if target_player.grid[flip_pos]:
                        # Calculate expected score change when this card is revealed
                        test_known = target_player.known.copy()
                        test_known[flip_pos] = True  # This card becomes known
                        test_score = expected_score_blind(target_player.grid, test_known, rank_probabilities, getattr(target_player, 'privately_visible', None))
                        ev = test_score - current_score
                        if ev < best_flip_ev:
                            best_flip_ev = ev
                            current_best_flip_position = flip_pos

                # Choose the better option: keep drawn card or discard and flip
                if best_draw_ev < best_flip_ev:
                    current_best_ev = best_draw_ev
                    current_action_type = "keep"
                    current_best_position = current_best_draw_position
                else:
                    current_best_ev = best_flip_ev
                    current_action_type = "flip"
                    current_best_position = current_best_flip_position

                # Track the overall best action across all possible draws
                if current_best_ev < best_overall_ev:
                    best_overall_ev = current_best_ev
                    best_action_type = current_action_type
                    if current_action_type == "keep":
                        best_draw_position = current_best_position
                        best_flip_position = None
                    else:
                        best_draw_position = None
                        best_flip_position = current_best_position

                # Weight by probability of drawing this card
                probability = count / total_remaining_cards
                draw_expected_value += current_best_ev * probability

    # --- Draw Advantage ---
    # Difference between draw and discard EVs (negative = draw is better)
    draw_advantage = draw_expected_value - discard_expected_value

    # --- Recommendation ---
    # If draw_advantage is negative, drawing is better; if positive, discard is better
    if draw_advantage < -0.5:
        recommendation = "Draw from deck"
    elif draw_advantage > 0.5:
        recommendation = "Take discard"
    else:
        # Dynamic: whichever EV is lower (more negative) is slightly preferred
        if draw_expected_value < discard_expected_value:
            recommendation = "Either action is similar (draw slightly preferred)"
        elif discard_expected_value < draw_expected_value:
            recommendation = "Either action is similar (discard slightly preferred)"
        else:
            recommendation = "Either action is similar"

    return {
        'draw_expected_value': round(draw_expected_value, 2),
        'discard_expected_value': round(discard_expected_value, 2),
        'recommendation': recommendation,
        'draw_advantage': round(draw_advantage, 2),
        'discard_card': f"{discard_card.rank}{discard_card.suit}",
        'discard_score': discard_card.score(),
        'current_hand_score': current_score,
        'best_discard_position': best_discard_position,
        'best_draw_position': best_draw_position,
        'best_flip_position': best_flip_position,
        'best_action_type': best_action_type,  # "keep" or "flip"
    }

def which_card_to_swap_for_discard(game, player=None):
    """if the player wants to swap the discard card, which card should they swap it with?"""
    # get the discard card
    discard_card = game.discard_pile[-1]
    # get the target player (or default to first player for backwards compatibility)
    target_player = player if player is not None else game.players[0]
    # get the available positions for the target player
    available_positions = [i for i, known in enumerate(target_player.known) if not known]
    # get the cards in the target player's grid
    cards_in_grid = [card for card in target_player.grid if card]
    # get the private deck counts
    private_deck_counts = get_private_deck_counts(game)

    # get the expected value of drawing from deck vs taking the discard card
    #loop through each card and use probabilities of the private deck counts to get the expected value of swapping the discard card with that card
    for card in cards_in_grid:
        # get the probability of the card being in the private deck
        probability = private_deck_counts[card.rank] / sum(private_deck_counts.values())
        # get the expected value of swapping the discard card with that card
        expected_value = probability * card.score()
        # add the expected value to the list
        expected_values.append(expected_value)
    # return the card with the highest expected value
    return cards_in_grid[expected_values.index(max(expected_values))]

def which_card_to_swap_for_deck(game):
    """if the player wants to swap the deck card, which card should they swap it with?"""
    # get the deck card
    deck_card = game.deck[-1]
    # get the human player
    human_player = game.players[0]
    # get the available positions for the human player
    available_positions = [i for i, known in enumerate(human_player.known) if not known]
    # get the cards in the human player's grid
    pass

def average_score_of_deck(game):
    """average score of the deck"""
    # get the deck
    deck = game.deck
    # get the average score of the deck
    return sum(card.score() for card in deck) / len(deck)



In [ ]:
import itertools
from collections import defaultdict
import random

class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def __str__(self):
        return f"{self.rank}{self.suit}"

    def __repr__(self):
        return self.__str__()

    def score(self):
        if self.rank == 'A':
            return 1
        elif self.rank == 'J':
            return 0
        elif self.rank in ['Q', 'K']:
            return 10
        else:
            return int(self.rank)

class GolfGame:
    RANKS = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
    SUITS = ['♠', '♥', '♦', '♣']

    def __init__(self):
        self.deck = self.create_deck()
        self.grid = [None, None, None, None]  # top-left, top-right, bottom-left, bottom-right
        self.known_cards = [False, False, True, True]  # initially know bottom 2 cards
        self.turn = 1
        self.discard_pile = []  # Complete history of discarded cards
        self.opponents_visible = []  # Cards visible in opponents' grids
        self.memory = {
            'all_seen_cards': [],  # Every card we've seen
            'discard_history': [],  # Full discard pile
            'cards_per_rank': {rank: 0 for rank in self.RANKS}  # Count of seen cards per rank
        }

    def create_deck(self):
        return [Card(rank, suit) for rank, suit in itertools.product(self.RANKS, self.SUITS)]

    def calculate_score(self, grid):
        """Calculate score for a 2x2 grid with pair cancellation"""
        scores = [card.score() if card else 0 for card in grid]
        total_score = sum(scores)

        # Check for pairs that cancel (any two matching ranks)
        ranks = [card.rank if card else None for card in grid]
        pairs = []
        used_positions = set()

        # Check all possible pairs (6 combinations for 4 positions)
        for pos1, pos2 in itertools.combinations(range(4), 2):
            if (ranks[pos1] and ranks[pos2] and
                ranks[pos1] == ranks[pos2] and
                pos1 not in used_positions and pos2 not in used_positions):
                pairs.append((pos1, pos2))
                used_positions.add(pos1)
                used_positions.add(pos2)
                total_score -= (scores[pos1] + scores[pos2])

        return total_score, pairs

    def update_memory(self, new_cards):
        """Update memory with newly seen cards"""
        for card in new_cards:
            if card and card not in self.memory['all_seen_cards']:
                self.memory['all_seen_cards'].append(card)
                self.memory['cards_per_rank'][card.rank] += 1

    def add_to_discard(self, card):
        """Add card to discard pile and update memory"""
        if card:
            self.memory['discard_history'].append(card)
            self.update_memory([card])

    def get_deck_probabilities(self, additional_seen_cards=None):
        """Calculate probability distribution of remaining cards in deck using full memory"""
        # Start with cards we've tracked in memory
        rank_counts = self.memory['cards_per_rank'].copy()

        # Add any additional cards passed in (for temporary calculations)
        if additional_seen_cards:
            for card in additional_seen_cards:
                if card:
                    rank_counts[card.rank] += 1

        # Calculate remaining cards for each rank (4 total per rank in deck)
        remaining_cards = {}
        for rank in self.RANKS:
            remaining_cards[rank] = max(0, 4 - rank_counts[rank])

        total_remaining = sum(remaining_cards.values())

        # Convert to probabilities
        probabilities = {}
        for rank in self.RANKS:
            probabilities[rank] = remaining_cards[rank] / total_remaining if total_remaining > 0 else 0

        return probabilities, total_remaining, rank_counts

    def expected_score_for_unknown_position(self, probabilities):
        """Calculate expected score for an unknown card position"""
        expected = 0
        for rank, prob in probabilities.items():
            card_score = Card(rank, '♠').score()  # suit doesn't matter for scoring
            expected += prob * card_score
        return expected

    def get_memory_analysis(self):
        """Get detailed analysis of what we've seen"""
        probs, total_remaining, seen_counts = self.get_deck_probabilities()

        analysis = {
            'total_cards_seen': len(self.memory['all_seen_cards']),
            'total_remaining_in_deck': total_remaining,
            'discard_pile_size': len(self.memory['discard_history']),
            'rank_analysis': {}
        }

        for rank in self.RANKS:
            remaining = 4 - seen_counts[rank]
            analysis['rank_analysis'][rank] = {
                'seen': seen_counts[rank],
                'remaining': remaining,
                'probability': probs[rank],
                'score': Card(rank, '♠').score()
            }

        return analysis

    def evaluate_take_discard_action(self, position, discard_card, current_grid, known_cards):
        """Evaluate taking discard card and placing it at position"""
        new_grid = current_grid.copy()
        new_grid[position] = discard_card
        new_known = known_cards.copy()
        new_known[position] = True

        # Calculate score with this new card
        known_score, pairs = self.calculate_score([card if new_known[i] else None for i, card in enumerate(new_grid)])

        # Add expected score for unknown positions using memory
        deck_probs, _, _ = self.get_deck_probabilities([discard_card])  # Include the card we're taking
        unknown_expected = 0
        for i in range(4):
            if not new_known[i]:
                unknown_expected += self.expected_score_for_unknown_position(deck_probs)

        total_expected = known_score + unknown_expected

        return {
            'action': f'Take {discard_card} → Position {position + 1}',
            'expected_score': total_expected,
            'known_score': known_score,
            'pairs': pairs,
            'position': position
        }

    def evaluate_draw_deck_action(self, position, current_grid, known_cards):
        """Evaluate drawing from deck and expected outcome at position"""
        deck_probs, total_remaining, _ = self.get_deck_probabilities()

        if total_remaining == 0:
            return {
                'action': f'Draw from deck → Position {position + 1}',
                'expected_score': float('inf'),  # No cards left
                'position': position,
                'error': 'No cards remaining in deck'
            }

        total_expected_score = 0
        best_cards = []

        for rank, prob in deck_probs.items():
            if prob == 0:
                continue

            # Simulate drawing this card
            drawn_card = Card(rank, '♠')  # suit doesn't matter
            new_grid = current_grid.copy()
            new_grid[position] = drawn_card
            new_known = known_cards.copy()
            new_known[position] = True

            # Calculate score
            known_score, pairs = self.calculate_score([card if new_known[i] else None for i, card in enumerate(new_grid)])

            # Add expected score for remaining unknown positions
            remaining_deck_probs, _, _ = self.get_deck_probabilities([drawn_card])
            unknown_expected = 0
            for i in range(4):
                if not new_known[i]:
                    unknown_expected += self.expected_score_for_unknown_position(remaining_deck_probs)

            total_score = known_score + unknown_expected
            total_expected_score += prob * total_score

            if drawn_card.score() <= 1:  # Good cards (A=1, J=0)
                best_cards.append((rank, prob))

        return {
            'action': f'Draw from deck → Position {position + 1}',
            'expected_score': total_expected_score,
            'position': position,
            'prob_good_card': sum(prob for rank, prob in best_cards),
            'best_possible': best_cards,
            'cards_remaining': total_remaining
        }

    def get_recommendations(self, grid, known_cards, discard_top, turn):
        """Get action recommendations with probabilities using full memory"""
        # Update memory with current known cards and discard top
        current_known = [card for i, card in enumerate(grid) if known_cards[i] and card]
        self.update_memory(current_known + [discard_top])

        current_score, current_pairs = self.calculate_score([card if known_cards[i] else None for i, card in enumerate(grid)])

        # Calculate baseline expected score (doing nothing)
        deck_probs, total_remaining, _ = self.get_deck_probabilities()
        baseline_unknown_expected = 0
        for i in range(4):
            if not known_cards[i]:
                baseline_unknown_expected += self.expected_score_for_unknown_position(deck_probs)
        baseline_expected = current_score + baseline_unknown_expected

        recommendations = []

        # Available positions (face-down cards only)
        available_positions = [i for i in range(4) if not known_cards[i]]

        if not available_positions:
            return {
                "message": "No moves available - all cards are face-up!",
                "baseline_score": current_score,
                "memory_analysis": self.get_memory_analysis()
            }

        # Evaluate taking discard card
        for pos in available_positions:
            eval_result = self.evaluate_take_discard_action(pos, discard_top, grid, known_cards)
            improvement = baseline_expected - eval_result['expected_score']
            confidence = min(95, max(5, 50 + improvement * 15))

            # Check if this creates a pair
            creates_pair = any(card and card.rank == discard_top.rank for i, card in enumerate(grid) if known_cards[i])

            recommendations.append({
                **eval_result,
                'improvement': improvement,
                'confidence': confidence,
                'type': 'take_discard',
                'creates_pair': creates_pair
            })

        # Evaluate drawing from deck
        for pos in available_positions:
            eval_result = self.evaluate_draw_deck_action(pos, grid, known_cards)
            if 'error' in eval_result:
                continue

            improvement = baseline_expected - eval_result['expected_score']
            confidence = min(85, max(10, 40 + improvement * 10))  # Lower confidence for unknown cards

            recommendations.append({
                **eval_result,
                'improvement': improvement,
                'confidence': confidence,
                'type': 'draw_deck'
            })

        # Sort by improvement (best first)
        recommendations.sort(key=lambda x: x['improvement'], reverse=True)

        return {
            'recommendations': recommendations,
            'baseline_score': baseline_expected,
            'current_known_score': current_score,
            'current_pairs': current_pairs,
            'turn': turn,
            'memory_analysis': self.get_memory_analysis()
        }

def main():
    """Example usage of the Golf solver with memory tracking"""
    game = GolfGame()

    # Simulate a game in progress - add some cards to memory
    # Cards that have been discarded throughout the game
    previous_discards = [
        Card('K', '♠'), Card('9', '♥'), Card('Q', '♦'), Card('8', '♣'), Card('A', '♠')
    ]

    for card in previous_discards:
        game.add_to_discard(card)

    # Example current game state
    grid = [
        None,  # top-left (unknown)
        Card('Q', '♠'),  # top-right (known, face-up)
        Card('6', '♥'),  # bottom-left (known from start)
        Card('J', '♣')   # bottom-right (known from start)
    ]

    known_cards = [False, True, True, True]
    discard_top = Card('6', '♠')  # Current top of discard pile
    turn = 2

    print("=== 4-CARD GOLF STRATEGY SOLVER (with Memory) ===\n")
    print("Current Grid:")
    print(f"[ {'?' if not known_cards[0] else str(grid[0])} | {grid[1] if known_cards[1] else '?'} ]")
    print(f"[ {grid[2] if known_cards[2] else '?'} | {grid[3] if known_cards[3] else '?'} ]")
    print(f"\nDiscard pile top: {discard_top}")
    print(f"Turn: {turn}/4\n")

    # Get recommendations
    result = game.get_recommendations(grid, known_cards, discard_top, turn)

    if 'message' in result:
        print(result['message'])
        print(f"Final score: {result['baseline_score']}")
        return

    # Show memory analysis
    memory = result['memory_analysis']
    print("=== MEMORY ANALYSIS ===")
    print(f"Cards seen: {memory['total_cards_seen']}")
    print(f"Cards remaining in deck: {memory['total_remaining_in_deck']}")
    print(f"Discard pile size: {memory['discard_pile_size']}")

    print("\nRank probabilities remaining:")
    for rank in ['A', 'J', '2', '6', 'Q', 'K']:  # Show key ranks
        info = memory['rank_analysis'][rank]
        print(f"  {rank}: {info['remaining']}/4 left ({info['probability']:.1%}) - Score: {info['score']}")

    print(f"\nCurrent known score: {result['current_known_score']}")
    if result['current_pairs']:
        pairs_str = ', '.join([f"Pos {p1+1} & {p2+1}" for p1, p2 in result['current_pairs']])
        print(f"Current pairs: {pairs_str}")
    print(f"Expected final score if no action: {result['baseline_score']:.1f}\n")

    print("RECOMMENDATIONS (best first):\n")

    for i, rec in enumerate(result['recommendations'][:5]):  # Show top 5
        rank = "🥇" if i == 0 else "🥈" if i == 1 else "🥉" if i == 2 else f"{i+1}."

        print(f"{rank} {rec['action']}")
        print(f"   Expected improvement: {rec['improvement']:+.1f} points")
        print(f"   Confidence: {rec['confidence']:.0f}%")
        print(f"   Expected final score: {rec['expected_score']:.1f}")

        if rec['type'] == 'take_discard' and rec.get('creates_pair'):
            print("   ⭐ CREATES A PAIR! ⭐")

        if rec['type'] == 'draw_deck':
            print(f"   Chance of good card (A, J): {rec['prob_good_card']:.1%}")
            if 'cards_remaining' in rec:
                print(f"   Cards left in deck: {rec['cards_remaining']}")

        print()

    # Strategy advice based on memory
    print("STRATEGY NOTES:")
    best_action = result['recommendations'][0]

    if best_action['improvement'] > 2:
        print("• Strong move available - high confidence recommendation")
    elif best_action['improvement'] > 0.5:
        print("• Decent improvement possible")
    elif best_action['improvement'] > -0.5:
        print("• Marginal decision - consider position and remaining turns")
    else:
        print("• No great options - might want to play conservatively")

    if turn >= 3:
        print("• Late in the game - prioritize certainty over potential")

    # Memory-based insights
    good_cards_left = sum(memory['rank_analysis'][rank]['remaining'] for rank in ['A', 'J'])
    total_left = memory['total_remaining_in_deck']
    if total_left > 0:
        good_card_prob = good_cards_left / total_left
        print(f"• {good_cards_left} good cards (A, J) left in {total_left} remaining cards ({good_card_prob:.1%})")

    # Check for pair opportunities
    if any(rec.get('creates_pair') for rec in result['recommendations'] if rec['type'] == 'take_discard'):
        print("• 🎯 PAIR OPPORTUNITY: Taking discard creates a matching pair!")

    # Check if specific ranks are depleted
    depleted_ranks = [rank for rank, info in memory['rank_analysis'].items() if info['remaining'] == 0]
    if depleted_ranks:
        print(f"• No more {', '.join(depleted_ranks)} cards available")

if __name__ == "__main__":
    main()

In [ ]:
import itertools
import random
import numpy as np
from collections import defaultdict, deque
import time
from typing import List, Dict, Tuple, Optional, Any
import json

class Card:
    def __init__(self, rank: str, suit: str):
        self.rank = rank
        self.suit = suit

    def __str__(self):
        return f"{self.rank}{self.suit}"

    def __repr__(self):
        return self.__str__()

    def score(self):
        if self.rank == 'A':
            return 1
        elif self.rank == 'J':
            return 0
        elif self.rank in ['Q', 'K']:
            return 10
        else:
            return int(self.rank)

    def __eq__(self, other):
        if not isinstance(other, Card):
            return False
        return self.rank == other.rank and self.suit == other.suit

    def __hash__(self):
        return hash((self.rank, self.suit))

class Player:
    def __init__(self, name, agent_type="random"):
        self.name = name
        self.agent_type = agent_type
        self.grid = [None] * 4  # 2x2 grid: [TL, TR, BL, BR]
        self.known = [False, False, True, True]  # Only bottom two known at start
        # Memory for tracking seen cards
        self.memory = {
            'all_seen_cards': [],
            'discard_history': [],
            'cards_per_rank': {rank: 0 for rank in ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']}
        }

    def reveal_all(self):
        self.known = [True] * 4

    def __str__(self):
        def show(i):
            return str(self.grid[i]) if self.known[i] else '?'
        return f"[ {show(0)} | {show(1)} ]\n[ {show(2)} | {show(3)} ]"

    def update_memory(self, new_cards):
        """Update memory with newly seen cards"""
        for card in new_cards:
            if card and card not in self.memory['all_seen_cards']:
                self.memory['all_seen_cards'].append(card)
                self.memory['cards_per_rank'][card.rank] += 1

    def add_to_discard_memory(self, card):
        """Add card to discard pile memory"""
        if card:
            self.memory['discard_history'].append(card)
            self.update_memory([card])

    def get_deck_probabilities(self, additional_seen_cards=None):
        """Calculate probability distribution of remaining cards in deck"""
        rank_counts = self.memory['cards_per_rank'].copy()

        if additional_seen_cards:
            for card in additional_seen_cards:
                if card:
                    rank_counts[card.rank] += 1

        remaining_cards = {}
        for rank in ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']:
            remaining_cards[rank] = max(0, 4 - rank_counts[rank])

        total_remaining = sum(remaining_cards.values())
        probabilities = {}
        for rank in ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']:
            probabilities[rank] = remaining_cards[rank] / total_remaining if total_remaining > 0 else 0

        return probabilities, total_remaining

    def expected_score_for_unknown_position(self, probabilities):
        """Calculate expected score for an unknown card position"""
        expected = 0
        for rank, prob in probabilities.items():
            card_score = Card(rank, '♠').score()
            expected += prob * card_score
        return expected

class RandomAgent:
    """Random agent that makes random legal moves"""
    def choose_action(self, player, game_state, trajectory=None):
        positions = [i for i, known in enumerate(player.known) if not known]
        if not positions:
            return None

        action = random.choice(['draw_deck', 'take_discard'])
        pos = random.choice(positions)

        if action == 'take_discard' and game_state.discard_pile:
            return {'type': 'take_discard', 'position': pos}
        else:
            # For draw_deck, also decide whether to keep the card
            keep = random.choice([True, False])
            return {'type': 'draw_deck', 'position': pos, 'keep': keep}

class HeuristicAgent:
    """Heuristic agent using strategy from original main.py"""
    def choose_action(self, player, game_state, trajectory=None):
        positions = [i for i, known in enumerate(player.known) if not known]
        if not positions:
            return None

        # Update memory with current known cards and discard top
        current_known = [card for i, card in enumerate(player.grid) if player.known[i] and card]
        discard_top = game_state.discard_pile[-1] if game_state.discard_pile else None
        player.update_memory(current_known + ([discard_top] if discard_top else []))

        # Calculate current score
        current_score = self.calculate_score([card if player.known[i] else None for i, card in enumerate(player.grid)])

        # Get deck probabilities
        deck_probs, total_remaining = player.get_deck_probabilities()

        # Calculate baseline expected score (doing nothing)
        baseline_unknown_expected = 0
        for i in range(4):
            if not player.known[i]:
                baseline_unknown_expected += player.expected_score_for_unknown_position(deck_probs)
        baseline_expected = current_score + baseline_unknown_expected

        best_action = None
        best_improvement = float('-inf')

        # Evaluate taking discard card
        if discard_top:
            for pos in positions:
                improvement = self.evaluate_take_discard_action(pos, discard_top, player, deck_probs, baseline_expected)
                if improvement > best_improvement:
                    best_improvement = improvement
                    best_action = {'type': 'take_discard', 'position': pos}

        # Evaluate drawing from deck
        for pos in positions:
            improvement = self.evaluate_draw_deck_action(pos, player, deck_probs, baseline_expected)
            if improvement > best_improvement:
                best_improvement = improvement
                best_action = {'type': 'draw_deck', 'position': pos, 'keep': True}

        # If no good action found, take discard if available, otherwise draw
        if not best_action:
            if discard_top:
                best_action = {'type': 'take_discard', 'position': random.choice(positions)}
            else:
                best_action = {'type': 'draw_deck', 'position': random.choice(positions), 'keep': True}

        return best_action

    def calculate_score(self, grid):
        """Calculate score for a grid (some cards might be None)"""
        scores = [card.score() if card else 0 for card in grid]
        total_score = sum(scores)

        ranks = [card.rank if card else None for card in grid]
        pairs = []
        used_positions = set()

        for pos1, pos2 in itertools.combinations(range(4), 2):
            if (ranks[pos1] and ranks[pos2] and
                ranks[pos1] == ranks[pos2] and
                pos1 not in used_positions and pos2 not in used_positions):
                pairs.append((pos1, pos2))
                used_positions.add(pos1)
                used_positions.add(pos2)
                total_score -= (scores[pos1] + scores[pos2])

        return total_score

    def evaluate_take_discard_action(self, position, discard_card, player, deck_probs, baseline_expected):
        """Evaluate taking discard card and placing it at position"""
        new_grid = player.grid.copy()
        new_grid[position] = discard_card
        new_known = player.known.copy()
        new_known[position] = True

        known_score = self.calculate_score([card if new_known[i] else None for i, card in enumerate(new_grid)])

        # Add expected score for unknown positions
        unknown_expected = 0
        for i in range(4):
            if not new_known[i]:
                unknown_expected += player.expected_score_for_unknown_position(deck_probs)

        total_expected = known_score + unknown_expected
        return baseline_expected - total_expected

    def evaluate_draw_deck_action(self, position, player, deck_probs, baseline_expected):
        """Evaluate drawing from deck and expected outcome at position"""
        total_expected_score = 0

        for rank, prob in deck_probs.items():
            if prob == 0:
                continue

            drawn_card = Card(rank, '♠')
            new_grid = player.grid.copy()
            new_grid[position] = drawn_card
            new_known = player.known.copy()
            new_known[position] = True

            known_score = self.calculate_score([card if new_known[i] else None for i, card in enumerate(new_grid)])

            unknown_expected = 0
            for i in range(4):
                if not new_known[i]:
                    unknown_expected += player.expected_score_for_unknown_position(deck_probs)

            total_score = known_score + unknown_expected
            total_expected_score += prob * total_score

        return baseline_expected - total_expected_score

class QLearningAgent:
    """Q-learning agent that actually learns from experience"""
    def __init__(self, learning_rate=0.1, discount_factor=0.9, epsilon=0.2):
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.training_mode = True

    def get_state_key(self, player, game_state):
        """Convert game state to a simplified string key for Q-table"""
        # Simplified state representation focusing on key features
        # Only track known cards and their scores, not specific suits
        known_cards = []
        for i, card in enumerate(player.grid):
            if player.known[i] and card:
                known_cards.append(card.rank)  # Only rank, not suit
            else:
                known_cards.append('?')

        # Sort known cards for consistency (same state regardless of position)
        known_cards_sorted = sorted([c for c in known_cards if c != '?'])
        unknown_count = known_cards.count('?')

        # Include discard top and round for context
        discard_top = game_state.discard_pile[-1].rank if game_state.discard_pile else 'None'

        return f"{known_cards_sorted}_{unknown_count}_{discard_top}_{game_state.round}"

    def get_action_key(self, action):
        """Convert action to a string key"""
        return f"{action['type']}_{action['position']}"

    def choose_action(self, player, game_state, trajectory=None):
        # Get legal actions first
        positions = [i for i, known in enumerate(player.known) if not known]
        if not positions:
            return None

        actions = []
        if game_state.discard_pile:
            for pos in positions:
                actions.append({'type': 'take_discard', 'position': pos})
        if game_state.deck:
            for pos in positions:
                actions.append({'type': 'draw_deck', 'position': pos, 'keep': True})

        if not actions:
            return None

        # Epsilon-greedy policy
        if self.training_mode and random.random() < self.epsilon:
            action = random.choice(actions)
        else:
            # Choose action with highest Q-value
            state_key = self.get_state_key(player, game_state)
            best_action = None
            best_value = float('-inf')

            for action in actions:
                action_key = self.get_action_key(action)
                q_value = self.q_table[state_key][action_key]
                if q_value > best_value:
                    best_value = q_value
                    best_action = action

            action = best_action or random.choice(actions)

        # Record action in trajectory for training
        if trajectory is not None:
            state_key = self.get_state_key(player, game_state)
            action_key = self.get_action_key(action)
            trajectory.append({
                'state_key': state_key,
                'action_key': action_key,
                'action': action
            })

        return action

    def update(self, state_key, action_key, reward, next_state_key, next_actions):
        """Update Q-values using Q-learning update rule"""
        max_next_q = 0
        if next_actions:
            max_next_q = max(self.q_table[next_state_key][self.get_action_key(a)]
                           for a in next_actions)

        current_q = self.q_table[state_key][action_key]
        new_q = current_q + self.learning_rate * (reward + self.discount_factor * max_next_q - current_q)
        self.q_table[state_key][action_key] = new_q

    def train_on_trajectory(self, trajectory, final_reward, final_score):
        """Train the agent on a complete game trajectory with improved rewards"""
        if not trajectory:
            return

        # Update Q-values for each step in the trajectory
        for i, step in enumerate(trajectory):
            state_key = step['state_key']
            action_key = step['action_key']

            # Calculate immediate reward for this action
            # Give small positive reward for taking actions (encourages exploration)
            # The main learning comes from the final reward
            immediate_reward = 0.1  # Small positive reward for taking action

            # Get next state and actions (if not the last step)
            if i < len(trajectory) - 1:
                next_step = trajectory[i + 1]
                next_state_key = next_step['state_key']
                next_actions = [next_step['action']]
            else:
                next_state_key = state_key  # Terminal state
                next_actions = []
                # Add final reward to the last action
                immediate_reward += final_reward

            self.update(state_key, action_key, immediate_reward, next_state_key, next_actions)

    def set_training_mode(self, training):
        """Enable or disable training mode"""
        self.training_mode = training

    def get_q_table_size(self):
        """Get the size of the Q-table for debugging"""
        total_entries = sum(len(actions) for actions in self.q_table.values())
        return len(self.q_table), total_entries

    def decay_epsilon(self, factor=0.995):
        """Decay epsilon for better exploration/exploitation balance"""
        self.epsilon = max(0.01, self.epsilon * factor)

class GolfGame:
    RANKS = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
    SUITS = ['♠', '♥', '♦', '♣']

    def __init__(self, num_players=4, agent_types=None, q_agents=None):
        self.num_players = num_players
        if agent_types is None:
            agent_types = ["random"] * num_players
        self.players = [Player(f'P{i+1}', agent_types[i]) for i in range(num_players)]
        self.agents = self.create_agents(agent_types, q_agents)
        self.deck = self.create_deck()
        self.discard_pile = []
        self.turn = 0  # Player index
        self.round = 1
        self.max_rounds = 4
        self.deal()

    def create_agents(self, agent_types, q_agents=None):
        agents = []
        for i, agent_type in enumerate(agent_types):
            if agent_type == "random":
                agents.append(RandomAgent())
            elif agent_type == "heuristic":
                agents.append(HeuristicAgent())
            elif agent_type == "qlearning":
                # Use persistent Q-learning agent if provided
                if q_agents and i < len(q_agents):
                    agents.append(q_agents[i])
                else:
                    agents.append(QLearningAgent())
            else:
                agents.append(RandomAgent())  # Default to random
        return agents

    def create_deck(self):
        return [Card(rank, suit) for rank, suit in itertools.product(self.RANKS, self.SUITS)]

    def deal(self):
        random.shuffle(self.deck)
        for player in self.players:
            for i in range(4):
                player.grid[i] = self.deck.pop()
        # Start discard pile
        self.discard_pile.append(self.deck.pop())

    def play_turn(self, player, trajectory=None):
        agent = self.agents[self.turn]
        action = agent.choose_action(player, self, trajectory)

        if not action:
            return  # No moves left

        if action['type'] == 'take_discard' and self.discard_pile:
            # Take from discard pile, swap with pos
            new_card = self.discard_pile.pop()
            old_card = player.grid[action['position']]
            player.grid[action['position']] = new_card
            player.known[action['position']] = True
            player.add_to_discard_memory(old_card)
            self.discard_pile.append(old_card)
        elif action['type'] == 'draw_deck' and self.deck:
            # Draw from deck
            new_card = self.deck.pop()
            if action.get('keep', True):
                old_card = player.grid[action['position']]
                player.grid[action['position']] = new_card
                player.known[action['position']] = True
                player.add_to_discard_memory(old_card)
                self.discard_pile.append(old_card)
            else:
                player.add_to_discard_memory(new_card)
                self.discard_pile.append(new_card)

    def all_players_done(self):
        return all(all(p.known) for p in self.players)

    def next_player(self):
        self.turn = (self.turn + 1) % self.num_players
        if self.turn == 0:
            self.round += 1

    def play_game(self, verbose=True, trajectories=None):
        if trajectories is None:
            trajectories = [None] * self.num_players

        # Each player must take exactly 4 turns, so game should last exactly 4 rounds
        while self.round <= self.max_rounds:
            player = self.players[self.turn]
            if verbose:
                print(f"\n-- {player.name}'s turn (Round {self.round}) --")
                print(f"Agent: {player.agent_type}")
                print(player)
                print(f"Top of discard: {self.discard_pile[-1]}")

            # Check if player has any moves available
            available_positions = [i for i, known in enumerate(player.known) if not known]
            if available_positions:
                self.play_turn(player, trajectories[self.turn])
            else:
                # Player has no moves (all cards face-up), but still counts as a turn
                if verbose:
                    print(f"{player.name} has no moves available (all cards face-up)")

            self.next_player()

        # Reveal all cards
        for p in self.players:
            p.reveal_all()
        if verbose:
            print("\n=== FINAL GRIDS ===")
            for p in self.players:
                print(f"{p.name} ({p.agent_type}):\n{p}\n")
        scores = [self.calculate_score(p.grid) for p in self.players]
        if verbose:
            for i, s in enumerate(scores):
                print(f"{self.players[i].name} ({self.players[i].agent_type}) score: {s}")
            winner_idx = scores.index(min(scores))
            print(f"Winner: {self.players[winner_idx].name} ({self.players[winner_idx].agent_type})")
        return scores

    def calculate_score(self, grid):
        scores = [card.score() if card else 0 for card in grid]
        total_score = sum(scores)
        ranks = [card.rank if card else None for card in grid]
        pairs = []
        used = set()
        for pos1, pos2 in itertools.combinations(range(4), 2):
            if (ranks[pos1] and ranks[pos2] and ranks[pos1] == ranks[pos2]
                and pos1 not in used and pos2 not in used):
                pairs.append((pos1, pos2))
                used.add(pos1)
                used.add(pos2)
                total_score -= (scores[pos1] + scores[pos2])
        return total_score

def run_simulations_with_training(num_games=1000, agent_types=None, verbose=False):
    """
    Run multiple simulations with Q-learning training

    Args:
        num_games: Number of games to simulate
        agent_types: List of agent types for each player
        verbose: Whether to print detailed output for each game

    Returns:
        Dictionary with simulation results and statistics
    """
    if agent_types is None:
        agent_types = ["random", "heuristic", "qlearning", "random"]

    num_players = len(agent_types)

    # Create persistent Q-learning agents
    q_agents = []
    for i, agent_type in enumerate(agent_types):
        if agent_type == "qlearning":
            q_agents.append(QLearningAgent(epsilon=0.2))  # Higher epsilon for more exploration
        else:
            q_agents.append(None)

    # Statistics tracking
    stats = {
        'total_games': num_games,
        'agent_types': agent_types,
        'wins_by_agent': defaultdict(int),
        'scores_by_agent': defaultdict(list),
        'average_scores': {},
        'win_rates': {},
        'score_distributions': defaultdict(list),
        'game_durations': [],
        'q_learning_progress': [],
        'learning_curves': defaultdict(list),  # Track scores over time
        'score_by_interval': defaultdict(list)  # Track average scores by intervals
    }

    print(f"Running {num_games} simulations with agents: {agent_types}")
    print("Q-learning agents will learn from experience!")

    # Track scores for learning curves
    interval_size = max(1, num_games // 20)  # 20 intervals for tracking

    for game_num in range(num_games):
        if verbose and game_num % 100 == 0:
            print(f"Game {game_num + 1}/{num_games}")

        # Create trajectories for Q-learning agents
        trajectories = []
        for i in range(num_players):
            if agent_types[i] == "qlearning":
                trajectories.append([])
            else:
                trajectories.append(None)

        # Create and play game
        game = GolfGame(num_players=num_players, agent_types=agent_types, q_agents=q_agents)
        scores = game.play_game(verbose=False, trajectories=trajectories)

        # Train Q-learning agents
        winner_idx = scores.index(min(scores))
        for i, agent_type in enumerate(agent_types):
            if agent_type == "qlearning" and trajectories[i]:
                # Calculate reward: stronger signals for learning
                if i == winner_idx:
                    reward = 10.0  # Big reward for winning
                else:
                    # Stronger negative reward based on score
                    # Lower scores should have higher rewards
                    if scores[i] <= 5:
                        reward = 2.0  # Good score
                    elif scores[i] <= 10:
                        reward = 0.0  # Average score
                    elif scores[i] <= 15:
                        reward = -2.0  # Bad score
                    else:
                        reward = -5.0  # Very bad score

                q_agents[i].train_on_trajectory(trajectories[i], reward, scores[i])

                # Decay epsilon for better learning
                if game_num % 100 == 0:  # Decay every 100 games
                    q_agents[i].decay_epsilon()

        # Record results
        winner_agent = agent_types[winner_idx]
        stats['wins_by_agent'][winner_agent] += 1

        # Record scores for each agent
        for i, score in enumerate(scores):
            agent_type = agent_types[i]
            stats['scores_by_agent'][agent_type].append(score)
            stats['score_distributions'][agent_type].append(score)

            # Track learning curves (every game)
            stats['learning_curves'][agent_type].append(score)

        # Record game duration (number of rounds)
        stats['game_durations'].append(game.round)

        # Track Q-learning progress and scores by intervals
        if game_num % 100 == 0 or game_num == num_games - 1:
            q_progress = {}
            for i, agent_type in enumerate(agent_types):
                if agent_type == "qlearning":
                    states, entries = q_agents[i].get_q_table_size()
                    q_progress[f"qlearning_{i}"] = {"states": states, "entries": entries}
            stats['q_learning_progress'].append((game_num, q_progress))

        # Track average scores by intervals
        if (game_num + 1) % interval_size == 0 or game_num == num_games - 1:
            interval_start = max(0, game_num - interval_size + 1)
            interval_end = game_num + 1

            for i, agent_type in enumerate(agent_types):
                if agent_type in stats['scores_by_agent']:
                    interval_scores = stats['scores_by_agent'][agent_type][interval_start:interval_end]
                    avg_score = np.mean(interval_scores)
                    stats['score_by_interval'][agent_type].append({
                        'interval': len(stats['score_by_interval'][agent_type]) + 1,
                        'games': f"{interval_start+1}-{interval_end}",
                        'avg_score': avg_score,
                        'min_score': min(interval_scores),
                        'max_score': max(interval_scores)
                    })

    # Calculate final statistics
    for agent_type in agent_types:
        if agent_type in stats['scores_by_agent']:
            scores = stats['scores_by_agent'][agent_type]
            stats['average_scores'][agent_type] = np.mean(scores)
            stats['win_rates'][agent_type] = stats['wins_by_agent'][agent_type] / num_games

    return stats

def run_simulations(num_games=1000, agent_types=None, verbose=False):
    """
    Run multiple simulations and collect statistics (without Q-learning training)

    Args:
        num_games: Number of games to simulate
        agent_types: List of agent types for each player
        verbose: Whether to print detailed output for each game

    Returns:
        Dictionary with simulation results and statistics
    """
    if agent_types is None:
        agent_types = ["random", "heuristic", "qlearning", "random"]

    num_players = len(agent_types)

    # Statistics tracking
    stats = {
        'total_games': num_games,
        'agent_types': agent_types,
        'wins_by_agent': defaultdict(int),
        'scores_by_agent': defaultdict(list),
        'average_scores': {},
        'win_rates': {},
        'score_distributions': defaultdict(list),
        'game_durations': []
    }

    print(f"Running {num_games} simulations with agents: {agent_types}")

    for game_num in range(num_games):
        if verbose and game_num % 100 == 0:
            print(f"Game {game_num + 1}/{num_games}")

        # Create and play game
        game = GolfGame(num_players=num_players, agent_types=agent_types)
        scores = game.play_game(verbose=False)

        # Record results
        winner_idx = scores.index(min(scores))
        winner_agent = agent_types[winner_idx]
        stats['wins_by_agent'][winner_agent] += 1

        # Record scores for each agent
        for i, score in enumerate(scores):
            agent_type = agent_types[i]
            stats['scores_by_agent'][agent_type].append(score)
            stats['score_distributions'][agent_type].append(score)

        # Record game duration (number of rounds)
        stats['game_durations'].append(game.round)

    # Calculate final statistics
    for agent_type in agent_types:
        if agent_type in stats['scores_by_agent']:
            scores = stats['scores_by_agent'][agent_type]
            stats['average_scores'][agent_type] = np.mean(scores)
            stats['win_rates'][agent_type] = stats['wins_by_agent'][agent_type] / num_games

    return stats

def print_simulation_results(stats):
    """Print formatted simulation results"""
    print("\n" + "="*60)
    print("SIMULATION RESULTS")
    print("="*60)
    print(f"Total games: {stats['total_games']}")
    print(f"Agents: {stats['agent_types']}")

    print("\nWIN RATES:")
    for agent_type in stats['agent_types']:
        win_rate = stats['win_rates'].get(agent_type, 0)
        wins = stats['wins_by_agent'].get(agent_type, 0)
        print(f"  {agent_type}: {win_rate:.2%} ({wins} wins)")

    print("\nAVERAGE SCORES:")
    for agent_type in stats['agent_types']:
        avg_score = stats['average_scores'].get(agent_type, 0)
        scores = stats['scores_by_agent'].get(agent_type, [])
        if scores:
            min_score = min(scores)
            max_score = max(scores)
            print(f"  {agent_type}: {avg_score:.2f} (range: {min_score}-{max_score})")

    print(f"\nAverage game duration: {np.mean(stats['game_durations']):.1f} rounds")

    # Show Q-learning progress if available
    if 'q_learning_progress' in stats and stats['q_learning_progress']:
        print("\nQ-LEARNING PROGRESS:")
        for game_num, progress in stats['q_learning_progress']:
            print(f"  Game {game_num}: {progress}")

    # Show learning curves and score intervals
    if 'score_by_interval' in stats:
        print("\nLEARNING CURVES (Score by Intervals):")
        for agent_type in stats['agent_types']:
            if agent_type in stats['score_by_interval'] and stats['score_by_interval'][agent_type]:
                print(f"\n  {agent_type.upper()} LEARNING PROGRESS:")
                intervals = stats['score_by_interval'][agent_type]

                # Show first few, middle, and last intervals
                to_show = []
                if len(intervals) <= 6:
                    to_show = intervals
                else:
                    to_show = intervals[:3] + intervals[len(intervals)//2-1:len(intervals)//2+1] + intervals[-3:]

                for interval in to_show:
                    print(f"    Interval {interval['interval']} (Games {interval['games']}): "
                          f"Avg={interval['avg_score']:.2f}, Range={interval['min_score']}-{interval['max_score']}")

                # Show overall improvement
                if len(intervals) >= 2:
                    first_avg = intervals[0]['avg_score']
                    last_avg = intervals[-1]['avg_score']
                    improvement = first_avg - last_avg
                    print(f"    Overall improvement: {improvement:+.2f} points "
                          f"({first_avg:.2f} → {last_avg:.2f})")

    # Show some interesting statistics
    print("\nDETAILED ANALYSIS:")
    for agent_type in stats['agent_types']:
        scores = stats['scores_by_agent'].get(agent_type, [])
        if scores:
            perfect_games = sum(1 for s in scores if s == 0)
            print(f"  {agent_type}: {perfect_games} perfect games (score = 0)")

            # Score distribution
            score_counts = defaultdict(int)
            for score in scores:
                score_counts[score] += 1
            most_common_score = max(score_counts.items(), key=lambda x: x[1])
            print(f"    Most common score: {most_common_score[0]} (occurred {most_common_score[1]} times)")

            # For Q-learning agents, show learning trend
            if agent_type == "qlearning" and 'learning_curves' in stats:
                learning_curve = stats['learning_curves'][agent_type]
                if len(learning_curve) >= 100:
                    first_100_avg = np.mean(learning_curve[:100])
                    last_100_avg = np.mean(learning_curve[-100:])
                    trend = first_100_avg - last_100_avg
                    print(f"    Learning trend: {trend:+.2f} points improvement "
                          f"({first_100_avg:.2f} → {last_100_avg:.2f})")

def plot_learning_curves(stats):
    """Plot learning curves for visualization (if matplotlib is available)"""
    try:
        import matplotlib.pyplot as plt

        plt.figure(figsize=(12, 8))

        for agent_type in stats['agent_types']:
            if agent_type in stats['learning_curves']:
                scores = stats['learning_curves'][agent_type]
                games = list(range(1, len(scores) + 1))

                # Plot individual scores with low alpha
                plt.scatter(games, scores, alpha=0.1, s=1, label=f'{agent_type} (individual)')

                # Plot moving average
                window_size = max(1, len(scores) // 50)  # 50 points for moving average
                if len(scores) >= window_size:
                    moving_avg = []
                    for i in range(len(scores)):
                        start = max(0, i - window_size // 2)
                        end = min(len(scores), i + window_size // 2 + 1)
                        moving_avg.append(np.mean(scores[start:end]))
                    plt.plot(games, moving_avg, linewidth=2, label=f'{agent_type} (moving avg)')

        plt.xlabel('Game Number')
        plt.ylabel('Score')
        plt.title('Learning Curves - Score vs Game Number')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.ylim(bottom=0)  # Scores can't be negative

        # Save plot
        plt.savefig('golf_learning_curves.png', dpi=300, bbox_inches='tight')
        print("\nLearning curves plot saved as 'golf_learning_curves.png'")
        plt.show()

    except ImportError:
        print("\nMatplotlib not available. Install with 'pip install matplotlib' to see learning curves plot.")

def main():
    print("=== GOLF GAME SIMULATION SUITE WITH Q-LEARNING ===")

    # Example 1: Single game with different agents
    print("\n1. Single game example:")
    agent_types = ["heuristic", "random", "qlearning", "random"]
    game = GolfGame(num_players=4, agent_types=agent_types)
    game.play_game(verbose=True)

    # Example 2: Run simulations with Q-learning training
    print("\n2. Running simulations with Q-learning training...")
    stats = run_simulations_with_training(num_games=1000, agent_types=agent_types, verbose=True)
    print_simulation_results(stats)

    # Example 3: Plot learning curves
    print("\n3. Plotting learning curves...")
    plot_learning_curves(stats)

    # Example 4: Compare trained vs untrained Q-learning
    print("\n4. Comparing trained vs untrained Q-learning:")

    # Untrained Q-learning
    print("\nUntrained Q-learning vs Random:")
    stats_untrained = run_simulations(num_games=20000, agent_types=["qlearning", "random"], verbose=False)
    print_simulation_results(stats_untrained)

    # Trained Q-learning
    print("\nTrained Q-learning vs Random:")
    stats_trained = run_simulations_with_training(num_games=20000, agent_types=["qlearning", "random"], verbose=False)
    print_simulation_results(stats_trained)

if __name__ == "__main__":
    main()

In [ ]:
from game import GolfGame

def play_human_vs_ai():
    print("=== HUMAN vs AI GOLF GAME ===")
    print("You will play against AI agents to test the gameplay rules.")
    print("Rules: All cards start face-down, each flip makes card public to all players")
    print("Tip: Enter 'q' during your turn to quit the game early.\n")

    # Choose opponent
    print("Choose your opponent:")
    print("1. Random Agent")
    print("2. Heuristic Agent")
    print("3. Q-Learning Agent")

    while True:
        try:
            choice = input("Enter 1, 2, or 3: ").strip()
            if choice == "1":
                opponent = "random"
                break
            elif choice == "2":
                opponent = "heuristic"
                break
            elif choice == "3":
                opponent = "qlearning"
                break
            else:
                print("Invalid choice! Enter 1, 2, or 3.")
        except:
            print("Invalid input! Please try again.")

    # Create game with human vs chosen AI
    agent_types = ["human", opponent]
    game = GolfGame(num_players=2, agent_types=agent_types)

    print(f"\nYou are playing against: {opponent.upper()} agent")
    print("You are Player 1 (P1)")
    print("Game starting...\n")

    # Play the game
    try:
        scores = game.play_game(verbose=True)

        print(f"\n=== GAME OVER ===")
        print(f"Your score: {scores[0]}")
        print(f"AI score: {scores[1]}")

        if scores[0] < scores[1]:
            print("🎉 YOU WIN! 🎉")
        elif scores[0] > scores[1]:
            print("😔 AI wins 😔")
        else:
            print("🤝 It's a tie! 🤝")

    except KeyboardInterrupt:
        print(f"\n=== GAME QUIT ===")
        print("Game was quit by player.")

def play_human_vs_multiple_ai():
    print("=== HUMAN vs MULTIPLE AI GOLF GAME ===")
    print("You will play against multiple AI agents in a 4-player game.")
    print("Rules: All cards start face-down, each flip makes card public to all players")
    print("Tip: Enter 'q' during your turn to quit the game early.\n")

    # Create 4-player game with human and 3 AI agents
    agent_types = ["human", "random", "heuristic", "qlearning"]
    game = GolfGame(num_players=4, agent_types=agent_types)

    print("You are Player 1 (P1)")
    print("Other players: Random, Heuristic, Q-Learning")
    print("Game starting...\n")

    # Play the game
    try:
        scores = game.play_game(verbose=True)

        print(f"\n=== GAME OVER ===")
        print(f"Your score: {scores[0]}")
        print(f"Random AI score: {scores[1]}")
        print(f"Heuristic AI score: {scores[2]}")
        print(f"Q-Learning AI score: {scores[3]}")

        winner_idx = scores.index(min(scores))
        if winner_idx == 0:
            print("🎉 YOU WIN! 🎉")
        else:
            print(f"😔 {agent_types[winner_idx].upper()} agent wins 😔")

    except KeyboardInterrupt:
        print(f"\n=== GAME QUIT ===")
        print("Game was quit by player.")

if __name__ == "__main__":
    print("Choose game mode:")
    print("1. Human vs 1 AI (2 players)")
    print("2. Human vs 3 AI (4 players)")

    while True:
        try:
            choice = input("Enter 1 or 2: ").strip()
            if choice == "1":
                play_human_vs_ai()
                break
            elif choice == "2":
                play_human_vs_multiple_ai()
                break
            else:
                print("Invalid choice! Enter 1 or 2.")
        except:
            print("Invalid input! Please try again.")

In [ ]:
<!-- Main HTML for Golf Card Game UI --> <!-- Entry point for the Golf Card Game web app -->
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Golf Card Game</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <style>
        body {
            font-family: Arial, sans-serif;
            max-width: none;
            margin: 0;
            padding: 0;
            background-color: #f0f0f0;
        }
        .container {
            background: white;
            padding: 12px 18px 16px 18px;
            border-radius: 14px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.10);
            width: 60%; /* 100% of the parent container */
            max-width: 900px;
            margin: 18px auto 18px auto;
            min-height: 0;
        }
        .game-setup {
            text-align: center;
            margin-bottom: 18px;
        }
        .game-board {
            display: none;
            margin-top: 0;
        }
        .player-grid {
            margin: 8px 0 6px 0;
            padding: 7px 6px 8px 6px;
            border: 2px solid #ddd;
            border-radius: 8px;
        }
        .four-player-grid {
            display: grid; /* 2x2 grid for 4 players */
            grid-template-columns: 1fr 1fr;
            gap: 12px;
        }
        .player-grid.current-turn {
            border-color: #007bff; /* Highlight for current turn */
            background-color: #f8f9fa;
            position: relative;
        }
        .player-grid.current-turn.turn-animate {
            animation: pulse-border 2.5s infinite; /* Border pulse animation */
        }
        @keyframes pulse-border {
            0% { box-shadow: 0 0 0 0 rgba(0,123,255,0.5); }
            70% { box-shadow: 0 0 0 8px rgba(0,123,255,0.1); }
            100% { box-shadow: 0 0 0 0 rgba(0,123,255,0.5); }
        }

        .grid-container {
            display: grid; /* 2x2 grid for cards */
            grid-template-columns: repeat(2, 1fr);
            gap: 10px;
            max-width: 300px;
            margin: 10px auto;
        }
        .card {
            width: 60px;
            height: 90px;
            border: 2px solid #333;
            border-radius: 8px;
            display: flex;
            align-items: center;
            justify-content: center;
            font-size: 18px;
            font-weight: bold;
            margin: 5px;
            cursor: pointer;
            position: relative;
            transition: all 0.2s ease;
        }
        .card.face-down {
            background: linear-gradient(45deg, #667eea, #764ba2); /* Blue gradient for face-down */
            color: white;
        }
        .card.face-up {
            background: white;
            color: #333;
        }
        .card.privately-visible {
            background: linear-gradient(45deg, #f39c12, #e67e22); /* Orange for private cards */
            color: white;
        }
        .card.public {
            border-color: #28a745; /* Green border for public cards */
            box-shadow: 0 0 5px #28a745;
        }
        /* Layered deck effect */
        .card.face-down#deckCard {
            background: linear-gradient(45deg, #667eea, #764ba2);
            box-shadow:
                0 2px 4px rgba(0,0,0,0.3),
                0 4px 8px rgba(0,0,0,0.2),
                0 6px 12px rgba(0,0,0,0.1);
            transform: translateZ(0);
            width: 70px;
            height: 100px;
            font-size: 20px;
        }

        /* Discard card styling */
        #discardCard {
            width: 70px;
            height: 100px;
            font-size: 16px;
        }
        .card.face-down#deckCard::before {
            content: '';
            position: absolute;
            top: 2px;
            left: 2px;
            right: 2px;
            bottom: 2px;
            background: linear-gradient(45deg, #667eea, #764ba2);
            border: 2px solid #333;
            border-radius: 6px;
            z-index: -1;
            box-shadow: 0 1px 3px rgba(0,0,0,0.2);
        }
        .card.face-down#deckCard::after {
            content: '';
            position: absolute;
            top: 4px;
            left: 4px;
            right: 4px;
            bottom: 4px;
            background: linear-gradient(45deg, #667eea, #764ba2);
            border: 2px solid #333;
            border-radius: 4px;
            z-index: -2;
            box-shadow: 0 1px 2px rgba(0,0,0,0.1);
        }
        .actions {
            text-align: center;
            margin: 20px 0;
        }
        .btn {
            padding: 10px 20px;
            margin: 5px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 16px;
        }
        .btn-primary {
            background: #007bff;
            color: white;
        }
        .btn-secondary {
            background: #6c757d;
            color: white;
        }
        .btn:hover {
            opacity: 0.8;
        }
        .btn:disabled {
            opacity: 0.5;
            cursor: not-allowed;
        }
        .game-info {
            background: #f8f9fa;
            border-radius: 10px;
            padding: 15px 25px;
            margin-bottom: 10px;
            font-size: 1.1em;
            font-weight: bold;
            text-align: center;
        }
        .modal {
            display: none;
            position: fixed;
            z-index: 1000;
            left: 0;
            top: 0;
            width: 100%;
            height: 100%;
            background-color: rgba(0,0,0,0.5); /* Overlay for modals */
        }
        .modal-content {
            background-color: white;
            margin: 0 auto;
            position: fixed;
            left: 50%;
            bottom: 40px;
            transform: translateX(-50%);
            padding: 16px 20px;
            border-radius: 10px;
            width: 320px;
            max-width: 90vw;
            text-align: center;
            z-index: 2000;
            box-shadow: 0 2px 12px rgba(0,0,0,0.18);
        }
        .position-buttons {
            display: grid;
            grid-template-columns: repeat(2, 1fr);
            grid-template-rows: repeat(2, 1fr);
            gap: 10px;
            margin-top: 15px;
            flex-wrap: wrap;
            justify-items: center;
            align-items: center;
        }
        .main-board {
            display: flex; /* Main layout: left, center, right panels */
            flex-direction: row;
            justify-content: flex-start;
            align-items: flex-start;
            gap: 10px;
            width: 100%;
        }
        .new-flex-layout {
            gap: 18px;
        }
        .left-panel {
            flex: 2;
            min-width: 120;
            margin-right: 0;
            padding-right: 4px;
        }
        .center-panel {
            flex: 1.2;
            min-width: 220px;
            max-width: 300px;
            display: flex;
            flex-direction: column;
            align-items: center;
            gap: 6px;
        }
        .right-panel.always-right-panel {
            flex: 0 0 260px;
            min-width: 240px;
            max-width: 320px;
            display: flex;
            flex-direction: column;
            align-items: flex-end;
            justify-content: flex-start;
            position: relative;
            z-index: 2;
            margin-left: 0;
        }
        .deck-discard-vertical {
            display: flex;
            flex-direction: column;
            align-items: center;
            gap: 7px;
        }
        .card:hover {
            box-shadow: 0 0 10px #007bff, 0 0 5px #764ba2;
            opacity: 0.92;
            cursor: pointer;
        }
        .card.disabled {
            opacity: 0.5;
            cursor: not-allowed;
        }
        .card.disabled:hover {
            box-shadow: none;
            opacity: 0.5;
        }
        .setup-and-board {
            display: flex;
            flex-direction: column;
            gap: 12px;
        }
        /* .legend { ... }  Legend styles commented out for now */
        /* Drag-and-drop highlight for grid cells */
        .card.drop-target {
            outline: 3px dashed #007bff; /* Highlight drop targets */
            box-shadow: 0 0 8px #007bff;
        }
        .card.not-droppable {
            cursor: not-allowed;
            opacity: 0.5;
        }
        /* Drawn card area */
        #drawnCardArea {
            display: none;
            flex-direction: column;
            align-items: center;
            margin-right: 20px;
        }
        #drawnCardDisplay {
            margin-bottom: 8px;
            width: 70px;
            height: 100px;
            font-size: 20px;
            display: flex;
            align-items: center;
            justify-content: center;
            border: 2px solid #333;
            border-radius: 8px;
            background: white;
            font-weight: bold;
            cursor: grab;
            user-select: none;
        }
        #drawnCardDisplay.dragging {
            opacity: 0.7;
            box-shadow: 0 0 10px #007bff;
        }
        #drawnCardDisplay.playable {
            outline: 3px solid #007bff;
            box-shadow: 0 0 12px #007bff;
            border-color: #007bff;
        }
        #discardCard.faded {
            opacity: 0.4;
            filter: grayscale(80%);
            border-style: dashed;
            cursor: not-allowed;
        }
        #probabilitiesPanelContainer {
            position: fixed; /* Always fixed to right */
            top: 40px;
            right: 20px;
            z-index: 100;
            min-width: 320px;
            max-width: 480px;
            width: 420px;
            display: flex;
            flex-direction: column;
            align-items: flex-end;
        }
        #probabilitiesPanel {
            margin-top: 0;
            margin-right: 0;
            min-width: 300px;
            max-width: 440px;
            font-size: 1.15em;
            align-self: flex-end;
            padding: 18px 24px;
        }
        .gameover-modal-inside {
            display: flex;
            align-items: center;
            justify-content: center;
            position: relative;
            width: 100%;
            margin-top: 40px;
            z-index: 10;
        }
        .modal-content-inside {
            background: white;
            border-radius: 12px;
            box-shadow: 0 2px 12px rgba(0,0,0,0.18);
            padding: 24px 18px 18px 18px;
            width: 90%;
            max-width: 260px;
            text-align: center;
        }
        #celebrationPanelContainer {
            position: fixed;
            top: 40px;
            left: 20px;
            z-index: 100;
            min-width: 420px;
            max-width: 600px;
            width: 520px;
            display: flex;
            flex-direction: column;
            align-items: flex-start;
        }
        #celebrationPanel {
            min-width: 380px;
            max-width: 560px;
            font-size: 1.15em;
            align-self: flex-start;
            padding: 24px 32px;
        }
    </style>
</head>
<body>
    <!-- Main container for the entire app --> <!-- App root -->
    <div class="container">
        <!-- Header Section (Title) --> <!-- Game title -->
        <div class="header">
            <h1>🏌️ Golf Card Game</h1>
        </div>
        <!-- Setup and Game Board Wrapper -->
        <div class="setup-and-board">
            <!-- Game Setup Panel (shown before game starts) -->
            <div class="game-setup" id="gameSetup">
                <h2>Game Setup</h2>
                <!-- Player Name Input -->
                <p>
                    <label for="playerName">Your Name:</label>
                    <input type="text" id="playerName" placeholder="Enter your name" style="margin-left:8px; padding:4px 8px; border-radius:4px; border:1px solid #ccc; font-size:1em;" /> <!-- Name input -->
                </p>
                <!-- Game Mode Dropdown -->
                <p>
                    <label>Game Mode:</label>
                    <select id="gameMode">
                        <option value="1v1">1 vs 1 AI</option>
                        <option value="1v3">1 vs 3 AI</option>
                    </select> <!-- Game mode selection -->
                </p>
                <!-- Opponent Type Dropdown (hidden for 1v3) -->
                <p id="opponentSection">
                    <label>Opponent Type:</label>
                    <select id="opponentType">
                        <option value="random">Random AI</option>
                        <option value="heuristic">Heuristic AI</option>
                        <option value="qlearning">Q-Learning AI</option>
                    </select> <!-- AI type selection -->
                </p>
                <!-- Number of Holes Dropdown -->
                <p>
                    <label for="numGames">Number of Holes:</label>
                    <select id="numGames" style="margin-left:8px; padding:4px 8px; border-radius:4px; border:1px solid #ccc; font-size:1em;">
                        <option value="1">1</option>
                        <option value="3">3</option>
                        <option value="6">6</option>
                        <option value="9">9</option>
                        <option value="18">18</option>
                    </select> <!-- Number of games/holes -->
                </p>
                <!-- Start/Restart Buttons -->
                <button onclick="startGame()" class="btn btn-primary">Start Game</button>
                <button onclick="restartGame()" class="btn btn-secondary" id="restartBtn" style="display:none;">Restart Game</button>
                <!-- Setup View Timer (shows how long bottom cards are visible) -->
                <div id="setupViewTimer" style="margin-top:16px;font-size:1.1em;color:#007bff;font-weight:bold;"></div>
            </div>
            <!-- Main Game Board (shown after game starts) -->
            <div class="game-board" id="gameBoard">
                <!-- Main Board Layout: Left, Center, Right Panels -->
                <div class="main-board new-flex-layout">
                    <!-- Left Panel: Game Info, Player Grids, Celebration GIFs, Match Summary -->
                    <div class="left-panel">
                        <!-- Game Info (current game, round, etc.) -->
                        <div class="game-info" id="gameInfo"></div>
                        <!-- Player Grids (all players' cards) -->
                        <div id="playerGrids"></div> <!-- Where all player grids are rendered -->
                    </div>
                    <!-- Center Panel: Deck, Discard, Drawn Card Area -->
                    <div class="center-panel">
                        <div style="display: flex; flex-direction: row; align-items: flex-end;">
                            <!-- Drawn Card Area (shows card drawn from deck) -->
                            <div id="drawnCardArea">
                                <div id="drawnCardDisplay" draggable="true"></div> <!-- Card drawn from deck -->
                            </div>
                            <!-- Deck and Discard Pile (vertical layout) -->
                            <div class="deck-discard-vertical">
                                <div>
                                    <h4>Deck</h4>
                                    <div class="card face-down" id="deckCard" title="Draw from Deck" onclick="drawFromDeck()"></div> <!-- Deck -->
                                    <div id="deckSize" style="text-align: center; margin-top: 5px; font-size: 12px;"></div>
                                </div>
                                <div>
                                    <h4>Discard</h4>
                                    <div class="card face-up" id="discardCard" title="Take Discard" onclick="takeDiscard()" draggable="true"></div> <!-- Discard pile -->
                                </div>
                            </div>
                        </div>
                    </div>
                    <!-- Right Panel: Probabilities Panel, Cumulative Score Chart -->
                    <div class="right-panel always-right-panel">
                        <div id="probabilitiesPanelContainer" style="width:100%;margin-top:auto;">
                            <!-- Probabilities Panel (deck composition, odds, etc.) -->
                            <div id="probabilitiesPanel"></div>
                            <!-- Cumulative Score Chart (line chart per round) -->
                            <div style="margin-top:32px;">
                                <canvas id="cumulativeScoreChart" height="180"></canvas>
                            </div>
                        </div>
                    </div>
                </div>
            </div>
        </div>
    </div>

    <!-- Position Selection Modal (for choosing where to place a card) -->
    <div id="positionModal" class="modal">
        <div class="modal-content">
            <h3 id="modalTitle">Select Position</h3>
            <p id="modalMessage">Choose a position:</p>
            <div id="positionButtons" class="position-buttons"></div> <!-- 2x2 grid of position buttons -->
        </div>
    </div>

    <!-- Celebration Panel (shows GIFs and match summary, left side, outside main container) -->
    <div id="celebrationPanelContainer">
        <div id="celebrationPanel"></div>
    </div>

    <!-- Main JavaScript for all UI/game logic --> <!-- All game logic below -->
    <script>
        let currentGameState = null; // Holds the current game state from backend
        let gameId = null; // Unique game session ID
        let drawnCard = null; // Card drawn from deck
        let currentAction = null; // Current action type
        let dragActive = false; // Drag state for discard
        let draggedDiscardCard = null; // Card being dragged from discard
        let drawnCardData = null; // Data for the currently drawn card
        let drawnCardDragActive = false; // Drag state for drawn card
        let turnAnimateTimeout = null; // Timeout for turn animation
        let lastTurnIndex = null; // Last turn index to detect turn changes
        let setupHideTimeout = null; // Timeout for hiding setup cards
        let setupCardsHidden = false; // Whether setup cards are hidden
        const SETUP_VIEW_SECONDS = 1.2; // Change this value for how long to show bottom cards
        let setupViewInterval = null; // Interval for setup view timer
        const SNAP_THRESHOLD = 110; // pixels

        // Celebration GIFs for human win (now loaded from JSON)
        let celebrationGifs = [];
        fetch('/static/golf_celebration_gifs.json')
            .then(response => response.json())
            .then(data => {
                if (data && data.data) {
                    celebrationGifs = data.data
                        .map(gif => gif.images && gif.images.downsized_medium && gif.images.downsized_medium.url)
                        .filter(Boolean);
                }
            });

        // Hide opponent selection for 1v3 mode
        document.getElementById('gameMode').addEventListener('change', function() {
            const opponentSection = document.getElementById('opponentSection');
            opponentSection.style.display = this.value === '1v1' ? 'block' : 'none';
        });

        function showSetupViewTimer(seconds) {
            const timerDiv = document.getElementById('setupViewTimer');
            timerDiv.textContent = `Bottom two cards visible for: ${seconds} second${seconds !== 1 ? 's' : ''}`;
        }

        function hideSetupViewTimer() {
            const timerDiv = document.getElementById('setupViewTimer');
            timerDiv.textContent = '';
        }

        async function startGame() {
            const gameMode = document.getElementById('gameMode').value;
            const opponentType = document.getElementById('opponentType').value;
            const playerName = document.getElementById('playerName').value || 'Human';
            const numGames = parseInt(document.getElementById('numGames').value) || 1;

            try {
                const response = await fetch('/create_game', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({
                        mode: gameMode,
                        opponent: opponentType,
                        player_name: playerName,
                        num_games: numGames
                    })
                });

                const data = await response.json();
                if (data.success && data.game_state && data.game_state.players) {
                    gameId = data.game_id;
                    currentGameState = data.game_state;
                    document.getElementById('gameSetup').style.display = 'none';
                    document.getElementById('gameBoard').style.display = 'block';
                    document.getElementById('restartBtn').style.display = 'inline-block';
                    setupCardsHidden = false;
                    if (setupHideTimeout) clearTimeout(setupHideTimeout);
                    if (setupViewInterval) clearInterval(setupViewInterval);
                    let secondsLeft = SETUP_VIEW_SECONDS;
                    showSetupViewTimer(secondsLeft);
                    setupViewInterval = setInterval(() => {
                        secondsLeft--;
                        if (secondsLeft > 0) {
                            showSetupViewTimer(secondsLeft);
                        } else {
                            hideSetupViewTimer();
                            clearInterval(setupViewInterval);
                        }
                    }, 1000);
                    setupHideTimeout = setTimeout(() => {
                        setupCardsHidden = true;
                        updateGameDisplay();
                        hideSetupViewTimer();
                        if (setupViewInterval) clearInterval(setupViewInterval);
                    }, SETUP_VIEW_SECONDS * 1000);
                    updateGameDisplay();
                } else {
                    console.error('Game start error:', data);
                    alert('Error starting game: ' + (data.error || 'Unexpected response from server. See console for details.'));
                }
            } catch (error) {
                console.error('Error starting game:', error);
                alert('Error starting game. Please try again.');
            }
        }

        async function refreshGameState() {
            if (!gameId) return;

            try {
                const response = await fetch(`/game_state/${gameId}`);
                const data = await response.json();
                if (data && !data.error) {
                    currentGameState = data;
                    updateGameDisplay();

                    if (data.game_over) {
                        // Game over - no modal needed
                    }
                }
            } catch (error) {
                console.error('Error refreshing game state:', error);
            }
        }

        function updateGameDisplay() {
            if (!currentGameState) return;

            // Get custom player names
            const playerNames = currentGameState.players.map(p => p.name);
            const currentPlayer = currentGameState.players[currentGameState.current_turn];
            const isHumanTurn = currentPlayer && currentGameState.current_turn === 0;

            // Show current game number and total games
            const roundDisplay = Math.min(currentGameState.round, currentGameState.max_rounds);
            let infoText = `Game ${currentGameState.current_game || 1} of ${currentGameState.num_games || 1} | Round ${roundDisplay}/${currentGameState.max_rounds}`;
            const celebrationPanel = document.getElementById('celebrationPanel');
            celebrationPanel.innerHTML = '';
            // Show AI thinking message if applicable
            if (currentGameState.ai_thinking) {
                celebrationPanel.innerHTML = `<div style="font-size:1.15em;font-weight:bold;text-align:center;background:#28a745;color:white;padding:10px 0 10px 0;border-radius:8px;margin-bottom:18px;">
                    AI is lining up its shot...</div>`;
            } else if (currentGameState.current_turn === 0 && !currentGameState.game_over) {
                celebrationPanel.innerHTML = `<div style="font-size:1.25em;font-weight:bold;text-align:center;background:#007bff;color:white;padding:10px 0 10px 0;border-radius:8px;margin-bottom:18px;">
                    Your Turn!</div>` + celebrationPanel.innerHTML;
            }
            if (currentGameState.game_over) {
                infoText += ' - Game Over!';
                // Show celebratory GIF in left panel if human wins
                let iconHtml = '';
                if (currentGameState.winner === 0) {
                    let gifUrl = '';
                    if (celebrationGifs.length > 0) {
                        gifUrl = celebrationGifs[Math.floor(Math.random() * celebrationGifs.length)];
                    }
                    iconHtml = gifUrl ? `<img src="${gifUrl}" alt="Golf Celebration" style="width:100%;max-width:320px;max-height:200px;display:block;margin:0 auto;border-radius:12px;box-shadow:0 2px 12px rgba(0,0,0,0.18);object-fit:contain;background:#fff;" />` : '';
                } else if (typeof currentGameState.winner === 'number') {
                    iconHtml = '🏆'; // AI wins
                }
                celebrationPanel.innerHTML = `<div style="font-size:1.25em;font-weight:bold;text-align:center;background:#007bff;color:white;padding:10px 0 10px 0;border-radius:8px;margin-bottom:18px;">Game Over</div><div style="text-align:center;margin-top:20px;">${iconHtml}</div>`;
            }

            document.getElementById('gameInfo').textContent = infoText;

            // Show match summary if match is over
            if (currentGameState.match_winner && currentGameState.current_game === currentGameState.num_games) {
                let summaryHtml = `<div style="background:#f8f9fa;padding:18px 24px;border-radius:12px;box-shadow:0 2px 12px #eee;margin-bottom:18px;max-width:480px;">
                    <h2 style="text-align:center;">Match Summary</h2>`;
                summaryHtml += `<div style="margin-bottom:10px;"><b>Final Cumulative Scores:</b></div><ul style="padding-left:18px;">`;
                for (let i = 0; i < currentGameState.players.length; i++) {
                    const winnerIcon = (Array.isArray(currentGameState.match_winner) && currentGameState.match_winner.includes(i)) ? ' 🏆' : '';
                    summaryHtml += `<li><b>${playerNames[i]}</b>: ${currentGameState.cumulative_scores[i]}${winnerIcon}</li>`;
                }
                summaryHtml += `</ul>`;
                if (Array.isArray(currentGameState.match_winner) && currentGameState.match_winner.length === 1) {
                    summaryHtml += `<div style="margin-top:12px;font-size:1.2em;text-align:center;color:#007bff;font-weight:bold;">Winner: ${playerNames[currentGameState.match_winner[0]]} 🏆</div>`;
                } else if (Array.isArray(currentGameState.match_winner)) {
                    summaryHtml += `<div style="margin-top:12px;font-size:1.2em;text-align:center;color:#007bff;font-weight:bold;">Winners: ${currentGameState.match_winner.map(i => playerNames[i]).join(', ')} 🏆</div>`;
                }
                summaryHtml += `</div>`;
                celebrationPanel.innerHTML = summaryHtml + celebrationPanel.innerHTML;
            }

            // Update deck size
            document.getElementById('deckSize').textContent = `${currentGameState.deck_size} cards`;

            // Update discard pile
            const discardCard = document.getElementById('discardCard');
            if (currentGameState.discard_top) {
                discardCard.textContent = `${currentGameState.discard_top.rank}${currentGameState.discard_top.suit}`;
            } else {
                discardCard.textContent = '?';
            }

            // Update player grids
            updatePlayerGrids();
            // Update probabilities panel
            updateProbabilitiesPanel();

            // Update cumulative score chart
            updateCumulativeScoreChart();
        }

        function updatePlayerGrids() {
            const container = document.getElementById('playerGrids');
            container.innerHTML = '';
            // Add or remove four-player-grid class
            if (currentGameState.players.length === 4) {
                container.classList.add('four-player-grid');
            } else {
                container.classList.remove('four-player-grid');
            }
            // Only clear timeout and remove animation if turn index changes
            const currentTurnIndex = currentGameState.current_turn;
            if (lastTurnIndex !== currentTurnIndex) {
                if (turnAnimateTimeout) {
                    clearTimeout(turnAnimateTimeout);
                    turnAnimateTimeout = null;
                }
                document.querySelectorAll('.player-grid.current-turn').forEach(el => el.classList.remove('turn-animate'));
            }
            currentGameState.players.forEach((player, index) => {
                const playerDiv = document.createElement('div');
                playerDiv.className = 'player-grid';
                if (index === currentGameState.current_turn) {
                    playerDiv.classList.add('current-turn'); // Highlight current turn
                }
                const isHuman = index === 0; // Human is always player 0
                const gridHtml = player.grid.map((card, pos) => {
                    if (!card) return '<div class="card face-down">?</div>'; // Empty slot
                    let cardClass = 'card';
                    let displayText = '?';
                    let extraAttrs = '';
                    if (isHuman && pos >= 2) {
                        if (!setupCardsHidden) {
                            cardClass += ' privately-visible'; // Show bottom cards at setup
                            displayText = `${card.rank}${card.suit}`;
                        } else if (card.public) {
                            cardClass += ' face-up public'; // Show if made public
                            displayText = `${card.rank}${card.suit}`;
                        } else {
                            cardClass += ' face-down'; // Hide after setup
                            displayText = '?';
                        }
                    } else if (card.visible) {
                        if (card.public) {
                            cardClass += ' face-up public';
                        } else {
                            cardClass += ' privately-visible';
                        }
                        displayText = `${card.rank}${card.suit}`;
                    } else {
                        cardClass += ' face-down';
                    }
                    // Drag-and-drop for human player
                    if (isHuman && !card.public && currentGameState.current_turn === 0 && !currentGameState.game_over) {
                        // Accept drop from discard or drawn card
                        extraAttrs += ' ondragover="event.preventDefault();this.classList.add(\'drop-target\');"';
                        extraAttrs += ' ondragleave="this.classList.remove(\'drop-target\');"';
                        extraAttrs += ` ondrop="handleDropOnGrid(${pos});this.classList.remove('drop-target');"`;
                        // If in flip mode, add flippable class
                        if (window.flipDrawnMode) {
                            cardClass += ' drop-target flippable';
                        }
                    } else if (isHuman) {
                        extraAttrs += ' class="card not-droppable"';
                    }
                    return `<div class="${cardClass}" data-position="${pos}" ${extraAttrs}>${displayText}</div>`;
                }).join('');
                // Show only the public score for each player, and private score for human
                let scoreText = '';
                let badgeHtml = '';
                if (currentGameState.public_scores && typeof currentGameState.public_scores[index] !== 'undefined') {
                    scoreText = ` - Score: ${currentGameState.public_scores[index]}`;
                    if (currentGameState.cumulative_scores && typeof currentGameState.cumulative_scores[index] !== 'undefined') {
                        scoreText += ` (Cumulative: ${currentGameState.cumulative_scores[index]})`;
                    }
                    if (currentGameState.game_over && index === currentGameState.winner) {
                        scoreText += ' 🏆';
                    }
                }
                playerDiv.innerHTML = `
                    <h3>${player.name} (${player.agent_type})${scoreText}</h3>
                    ${badgeHtml}
                    <div class="grid-container">${gridHtml}</div>
                `;
                container.appendChild(playerDiv);
            });
            // Robust delayed turn animation for human (border pulse)
            if (currentTurnIndex === 0 && !currentGameState.game_over && lastTurnIndex !== currentTurnIndex) {
                turnAnimateTimeout = setTimeout(() => {
                    // Only add animation if still human's turn
                    if (currentGameState.current_turn === 0 && !currentGameState.game_over) {
                        const grids = document.querySelectorAll('.player-grid.current-turn');
                        if (grids.length > 0) {
                            grids[0].classList.add('turn-animate'); // Border pulse
                        }
                    }
                }, 1); // Delay for border pulse (set to 1ms for instant)
            }
            lastTurnIndex = currentTurnIndex;
            // Attach click handler to flippable cards in flip mode
            if (window.flipDrawnMode) {
                document.querySelectorAll('.flippable').forEach(el => {
                    el.onclick = function() {
                        const pos = parseInt(this.getAttribute('data-position'));
                        flipDrawnCardOnGrid(pos);
                    };
                });
            }
        }

        async function takeDiscard() {
            // Check if the discard card is disabled
            const discardCard = document.getElementById('discardCard');
            if (discardCard.classList.contains('disabled')) {
                return; // Do nothing if disabled
            }

            const availablePositions = getAvailablePositions();
            if (availablePositions.length === 0) {
                alert('No available positions!');
                return;
            }

            if (!currentGameState.discard_top) {
                alert('No discard pile available!');
                return;
            }

            showPositionModal('Take Discard Card', 'Choose position to place the discard card:', availablePositions, 'take_discard');
        }

        async function drawFromDeck() {
            // Check if the deck card is disabled
            const deckCard = document.getElementById('deckCard');
            if (deckCard.classList.contains('disabled')) {
                return; // Do nothing if disabled
            }

            const availablePositions = getAvailablePositions();
            if (availablePositions.length === 0) {
                alert('No available positions!');
                return;
            }

            if (currentGameState.deck_size === 0) {
                alert('No cards left in deck!');
                return;
            }

            try {
                const response = await fetch(`/draw_card/${gameId}`);
                const data = await response.json();

                if (data.success) {
                    showDrawnCardArea(data.drawn_card);
                } else {
                    alert('Error drawing card: ' + (data.error || 'Unknown error'));
                }
            } catch (error) {
                console.error('Error drawing card:', error);
                alert('Error drawing card. Please try again.');
            }
        }

        function showDrawnCardArea(card) {
            drawnCardData = card;
            const area = document.getElementById('drawnCardArea');
            const display = document.getElementById('drawnCardDisplay');
            display.textContent = `${card.rank}${card.suit}`;
            area.style.display = 'flex';
            display.setAttribute('draggable', 'true');
            display.classList.remove('dragging');
            display.classList.add('playable');
            // Fade the discard card and make it unclickable
            const discardCard = document.getElementById('discardCard');
            discardCard.classList.add('faded');
            discardCard.onclick = null;
            discardCard.setAttribute('tabindex', '-1');
            window.flipDrawnMode = true; // Enable flip mode immediately
            updatePlayerGrids();
        }

        function hideDrawnCardArea() {
            drawnCardData = null;
            drawnCardDragActive = false;
            document.getElementById('drawnCardArea').style.display = 'none';
            document.getElementById('drawnCardDisplay').classList.remove('playable');
            const discardCard = document.getElementById('discardCard');
            discardCard.classList.remove('faded');
            discardCard.onclick = takeDiscard;
            discardCard.setAttribute('tabindex', '0');
            window.flipDrawnMode = false;
        }

        function getAvailablePositions() {
            if (!currentGameState || currentGameState.current_turn !== 0) return [];

            const humanPlayer = currentGameState.players[0];
            const positions = [];

            for (let i = 0; i < humanPlayer.grid.length; i++) {
                const card = humanPlayer.grid[i];
                if (card && !card.public) {  // Card exists but not publicly known
                    positions.push(i);
                }
            }

            return positions;
        }

        function showPositionModal(title, message, positions, actionType) {
            currentAction = actionType;
            document.getElementById('modalTitle').textContent = title;
            document.getElementById('modalMessage').textContent = message;

            const buttonsContainer = document.getElementById('positionButtons');
            buttonsContainer.innerHTML = '';

            // Get the human player's grid
            const humanPlayer = currentGameState.players[0];

            // Create a 2x2 grid of buttons (positions 0-3)
            for (let row = 0; row < 2; row++) {
                for (let col = 0; col < 2; col++) {
                    const pos = row * 2 + col;
                    if (positions.includes(pos)) {
                        let cardLabel = '?';
                        const card = humanPlayer.grid[pos];
                        if (card) {
                            if (card.visible) {
                                cardLabel = `${card.rank}${card.suit}`;
                            } else {
                                cardLabel = '?';
                            }
                        }
                        const button = document.createElement('button');
                        button.className = 'btn btn-primary';
                        button.textContent = cardLabel;
                        button.onclick = () => executeAction(pos, actionType);
                        buttonsContainer.appendChild(button);
                    } else {
                        // Add an invisible placeholder to keep grid shape
                        const placeholder = document.createElement('div');
                        placeholder.style.visibility = 'hidden';
                        buttonsContainer.appendChild(placeholder);
                    }
                }
            }

            document.getElementById('positionModal').style.display = 'block';
        }

        async function executeAction(position, actionType = null) {
            document.getElementById('positionModal').style.display = 'none';

            const action = {
                game_id: gameId,
                action: {
                    type: actionType || currentAction,
                    position: position
                }
            };

            // Add extra fields based on action type
            if (actionType === 'draw_keep' || currentAction === 'draw_keep') {
                action.action.type = 'draw_deck';
                action.action.keep = true;
            } else if (actionType === 'draw_discard' || currentAction === 'draw_discard') {
                action.action.type = 'draw_deck';
                action.action.keep = false;
                action.action.flip_position = position;
            }

            try {
                const response = await fetch('/make_move', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify(action)
                });

                const data = await response.json();
                if (data.success) {
                    currentGameState = data.game_state;
                    updateGameDisplay();
                    // Immediately refresh to catch any AI moves or turn changes
                    refreshGameState();
                    if (data.game_state.game_over) {
                        // Game over - no modal needed
                    }
                } else {
                    console.error('Action failed:', data.error);
                    alert('Action failed: ' + (data.error || 'Unknown error'));
                }
            } catch (error) {
                console.error('Error executing action:', error);
                alert('Error executing action. Please try again.');
            }
        }

        function restartGame() {
            currentGameState = null;
            gameId = null;
            document.getElementById('gameBoard').style.display = 'none';
            document.getElementById('gameSetup').style.display = 'block';
            document.getElementById('restartBtn').style.display = 'none';
            setupCardsHidden = false;
            if (setupHideTimeout) clearTimeout(setupHideTimeout);
            if (setupViewInterval) clearInterval(setupViewInterval);
            showSetupViewTimer(SETUP_VIEW_SECONDS);
            // Clear celebration panel on restart
            const celebrationPanel = document.getElementById('celebrationPanel');
            celebrationPanel.innerHTML = '';
        }

        // Close modals when clicking outside
        window.onclick = function(event) {
            const modals = document.getElementsByClassName('modal');
            for (let modal of modals) {
                if (event.target === modal) {
                    modal.style.display = 'none';
                }
            }
        }

        // Periodically refresh game state to catch AI moves
        setInterval(() => {
            if (gameId && currentGameState && !currentGameState.game_over) {
                refreshGameState();
            }
        }, 1000);

        // Drawn card drag-and-drop logic
        document.addEventListener('DOMContentLoaded', () => {
            const discardCard = document.getElementById('discardCard');
            discardCard.addEventListener('dragstart', (e) => {
                dragActive = true;
                discardCard.classList.add('drop-target');
                // Store the discard card value at drag start
                if (currentGameState && currentGameState.discard_top) {
                    draggedDiscardCard = {
                        rank: currentGameState.discard_top.rank,
                        suit: currentGameState.discard_top.suit
                    };
                } else {
                    draggedDiscardCard = null;
                }
            });
            discardCard.addEventListener('dragend', (e) => {
                dragActive = false;
                discardCard.classList.remove('drop-target');
                draggedDiscardCard = null;
                // Remove highlight from all grid cells
                document.querySelectorAll('.card.drop-target').forEach(el => el.classList.remove('drop-target'));
            });
            // Drawn card drag logic
            const display = document.getElementById('drawnCardDisplay');
            display.addEventListener('dragstart', (e) => {
                if (!drawnCardData) return e.preventDefault();
                drawnCardDragActive = true;
                display.classList.add('dragging');
            });
            display.addEventListener('dragend', (e) => {
                drawnCardDragActive = false;
                display.classList.remove('dragging');
                document.querySelectorAll('.card.drop-target').forEach(el => el.classList.remove('drop-target'));
            });
        });

        // Helper to animate snap effect
        function animateSnapToGrid(cardElem, targetElem, callback) {
            const cardRect = cardElem.getBoundingClientRect();
            const targetRect = targetElem.getBoundingClientRect();
            // Create a clone
            const clone = cardElem.cloneNode(true);
            document.body.appendChild(clone);
            clone.style.position = 'fixed';
            clone.style.left = cardRect.left + 'px';
            clone.style.top = cardRect.top + 'px';
            clone.style.width = cardRect.width + 'px';
            clone.style.height = cardRect.height + 'px';
            clone.style.zIndex = 9999;
            clone.style.pointerEvents = 'none';
            clone.style.transition = 'all 0.2s cubic-bezier(.4,1.4,.6,1)';
            // Hide original
            cardElem.style.visibility = 'hidden';
            // Animate to target
            requestAnimationFrame(() => {
                clone.style.left = targetRect.left + 'px';
                clone.style.top = targetRect.top + 'px';
            });
            // After animation, remove clone and callback
            setTimeout(() => {
                clone.remove();
                cardElem.style.visibility = '';
                if (callback) callback();
            }, 200);
        }

        function handleDropOnGrid(pos) {
            // Find the grid cell element
            const gridCells = document.querySelectorAll('.player-grid.current-turn .grid-container .card');
            let targetElem = null;
            gridCells.forEach(cell => {
                if (parseInt(cell.getAttribute('data-position')) === pos) {
                    targetElem = cell;
                }
            });
            // Drawn card drop
            if (drawnCardDragActive && drawnCardData) {
                const drawnCardElem = document.getElementById('drawnCardDisplay');
                if (targetElem && drawnCardElem) {
                    animateSnapToGrid(drawnCardElem, targetElem, () => {
                        executeAction(pos, 'draw_keep');
                        hideDrawnCardArea();
                    });
                } else {
                    executeAction(pos, 'draw_keep');
                    hideDrawnCardArea();
                }
                return;
            }
            // Discard card drop
            if (!dragActive) return;
            if (currentGameState.current_turn !== 0 || currentGameState.game_over || !currentGameState.discard_top) return;
            const card = currentGameState.players[0].grid[pos];
            if (!card || card.public) return;
            if (!draggedDiscardCard ||
                currentGameState.discard_top.rank !== draggedDiscardCard.rank ||
                currentGameState.discard_top.suit !== draggedDiscardCard.suit) {
                alert('The discard card has changed. Please try again.');
                return;
            }
            const discardCardElem = document.getElementById('discardCard');
            if (targetElem && discardCardElem) {
                animateSnapToGrid(discardCardElem, targetElem, () => {
                    executeAction(pos, 'take_discard');
                });
            } else {
                executeAction(pos, 'take_discard');
            }
        }

        function flipDrawnCardOnGrid(pos) {
            if (!window.flipDrawnMode || !drawnCardData) return;
            // Only allow if the position is not public
            const card = currentGameState.players[0].grid[pos];
            if (!card || card.public) return;
            // Discard the drawn card and flip this position
            executeAction(pos, 'draw_discard');
            hideDrawnCardArea();
        }

        function updateProbabilitiesPanel() {
            const panel = document.getElementById('probabilitiesPanel');
            if (!currentGameState || !currentGameState.probabilities) {
                panel.innerHTML = '';
                return;
            }
            const probs = currentGameState.probabilities;
            let html = '<div style="background:#f8f9fa;padding:12px 18px;border-radius:10px;box-shadow:0 1px 4px #eee;">';
            html += '<h4 style="margin-top:0;">Probabilities</h4>';

            // Expected Value Comparison (most important - show first)
            if (probs.expected_value_draw_vs_discard && currentGameState.current_turn === 0 && !currentGameState.game_over) {
                const ev = probs.expected_value_draw_vs_discard;
                html += '<div style="margin-bottom:12px;padding:8px 12px;background:#e8f4fd;border-radius:6px;border-left:4px solid #007bff;">';
                html += '<div style="font-weight:bold;color:#007bff;margin-bottom:4px;">🎯 Strategic Recommendation:</div>';
                html += `<div style="font-size:14px;margin-bottom:2px;"><b>${ev.recommendation}</b></div>`;
                html += `<div style="font-size:12px;color:#666;">Draw: +${ev.draw_expected_value} | Discard: +${ev.discard_expected_value} | Advantage: ${ev.draw_advantage > 0 ? '+' : ''}${ev.draw_advantage}</div>`;
                if (ev.discard_card) {
                    html += `<div style="font-size:12px;color:#666;">Discard: ${ev.discard_card} (score: ${ev.discard_score})</div>`;
                }
                html += '</div>';
            }

            // Deck composition
            if (probs.deck_counts) {
                // Desired order: J, A, 2-10, Q, K
                const order = ['J', 'A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'Q', 'K'];
                html += '<div><b>(Unknown Cards left):</b>';
                html += '<table style="font-size:13px;margin-top:4px;margin-bottom:6px;width:50%;border-collapse:collapse;">';
                html += '<thead><tr><th style="text-align:left;padding-right:10px;">Rank</th><th style="text-align:left;">Count</th></tr></thead><tbody>';
                for (const rank of order) {
                    if (probs.deck_counts[rank] !== undefined) {
                        html += `<tr><td style="padding-right:0px;">${rank}</td><td>${probs.deck_counts[rank]}</td></tr>`;
                    }
                }
                html += '</tbody></table></div>';
            }

            // Probability of drawing a pair (for human)
            if (probs.prob_draw_pair && probs.prob_draw_pair.length > 0) {
                html += `<div style="margin-top:4px;">`;
                html += `<b>Prob. next card matches your grid:</b> <span style="color:#007bff;">${probs.prob_draw_pair[0]}</span>`;
                html += '</div>';
            }
            // Probability of drawing a card that improves your hand (for human)
            if (probs.prob_improve_hand && probs.prob_improve_hand.length > 0) {
                html += `<div style="margin-top:4px;">`;
                html += `<b>Prob. next card improves your hand:</b> <span style="color:#007bff;">${probs.prob_improve_hand[0]}</span>`;
                html += '</div>';
            }
            html += '</div>';
            panel.innerHTML = html;
        }

        // Draw a line chart of cumulative scores for all players
        let cumulativeScoreChart = null;
        let lastChartGameId = null;
        let lastChartRound = null;
        function updateCumulativeScoreChart() {
            const ctx = document.getElementById('cumulativeScoreChart').getContext('2d');
            if (!currentGameState || !currentGameState.cumulative_scores || !currentGameState.current_game) return;
            // Build score history for each player, per round
            if (!window.cumulativeScoreHistory || lastChartGameId !== gameId) {
                // Reset history if new game session
                window.cumulativeScoreHistory = [];
                for (let i = 0; i < currentGameState.players.length; i++) {
                    window.cumulativeScoreHistory.push([]);
                }
                window.cumulativeScoreLabels = [];
                lastChartGameId = gameId;
                lastChartRound = null;
            }
            // Always add a point for the first round of the first game if not present
            if (window.cumulativeScoreLabels.length === 0 && currentGameState.round === 1) {
                for (let i = 0; i < currentGameState.players.length; i++) {
                    window.cumulativeScoreHistory[i].push(currentGameState.cumulative_scores[i]);
                }
                window.cumulativeScoreLabels.push('G1R1');
                lastChartRound = 'G1R1';
            }
            // Only update if new round or game over
            const roundKey = `G${currentGameState.current_game}R${currentGameState.round}`;
            if (window.cumulativeScoreLabels[window.cumulativeScoreLabels.length - 1] !== roundKey && (currentGameState.round > 1 || currentGameState.current_game > 1 || currentGameState.game_over)) {
                for (let i = 0; i < currentGameState.players.length; i++) {
                    window.cumulativeScoreHistory[i].push(currentGameState.cumulative_scores[i]);
                }
                window.cumulativeScoreLabels.push(roundKey);
                lastChartRound = roundKey;
            } else if (window.cumulativeScoreLabels[window.cumulativeScoreLabels.length - 1] === roundKey) {
                // No new round, don't update chart
                return;
            }
            // X axis: per round (G1R1, G1R2, ...)
            const labels = window.cumulativeScoreLabels;
            // Colors for each player
            const colors = ['#007bff', '#e67e22', '#28a745', '#764ba2', '#f39c12', '#e74c3c'];
            const datasets = currentGameState.players.map((player, i) => ({
                label: player.name,
                data: window.cumulativeScoreHistory[i],
                borderColor: colors[i % colors.length],
                backgroundColor: colors[i % colors.length],
                fill: false,
                tension: 0.2,
                pointRadius: 1,
                pointHoverRadius: 5
            }));
            if (cumulativeScoreChart) {
                cumulativeScoreChart.data.labels = labels;
                cumulativeScoreChart.data.datasets = datasets;
                cumulativeScoreChart.update();
            } else {
                cumulativeScoreChart = new Chart(ctx, {
                    type: 'line',
                    data: {
                        labels: labels,
                        datasets: datasets
                    },
                    options: {
                        responsive: true,
                        plugins: {
                            legend: { display: true, position: 'bottom' },
                            title: { display: true, text: 'Cumulative Scores by Round' }
                        },
                        scales: {
                            x: {
                                title: { display: true, text: 'Game & Round' },
                                ticks: { autoSkip: false }
                            },
                            y: {
                                title: { display: true, text: 'Score' },
                                beginAtZero: true
                            }
                        }
                    }
                });
            }
        }

        // Show timer on initial load
        document.addEventListener('DOMContentLoaded', function() {
            showSetupViewTimer(SETUP_VIEW_SECONDS);
        });

        function onDrop(card, slot) {
            const cardRect = card.getBoundingClientRect();
            const slotRect = slot.getBoundingClientRect();

            const dx = cardRect.left - slotRect.left;
            const dy = cardRect.top - slotRect.top;
            const distance = Math.sqrt(dx * dx + dy * dy);

            if (distance < SNAP_THRESHOLD) {
                // Snap the card to the slot
                card.style.transition = 'all 0.2s';
                card.style.left = `${slotRect.left}px`;
                card.style.top = `${slotRect.top}px`;
                // Optionally, update your game state here
            } else {
                // Return card to original position or handle as invalid drop
            }
        }
    </script>
</body>
</html>

In [ ]:
from collections import Counter

def get_deck_counts(game):
    """Return a dict of rank -> count for all cards left in the deck."""
    return dict(Counter(card.rank for card in game.deck))

def prob_draw_lower_than_min_faceup(game):
    """For each player, probability that next card is lower than their lowest card (face-up or not)."""
    results = []
    deck = game.deck
    if not deck:
        return ['0.0%' for _ in game.players]
    for player in game.players:
        player_cards = [card for card in player.grid if card]
        if not player_cards:
            results.append('0.0%')
            continue
        min_val = min(card.score() for card in player_cards)
        lower = [card for card in deck if card.score() < min_val]
        prob = len(lower) / len(deck)
        results.append(f'{round(prob * 100, 1)}%')
    return results




def prob_draw_pair(game):
    """For each player, probability that next card matches any rank in their grid."""
    results = []
    deck = game.deck
    if not deck:
        return ['0.0%' for _ in game.players]
    for player in game.players:
        ranks_in_grid = set(card.rank for card in player.grid if card)
        matching = [card for card in deck if card.rank in ranks_in_grid]
        prob = len(matching) / len(deck)
        results.append(f'{round(prob * 100, 1)}%')
    return results


def prob_improve_hand(game):
    """
    For each player, return the probability that drawing the next card would improve their hand,
    either by:
    - Forming a pair with any card in their grid, or
    - Being lower than any card in their grid (for a potential swap).
    """
    results = []
    deck = game.deck
    if not deck:
        return ['0.0%' for _ in game.players]

    for player in game.players:
        player_cards = [card for card in player.grid if card]
        if not player_cards:
            results.append('0.0%')
            continue

        all_ranks = set(card.rank for card in player_cards)
        all_scores = [card.score() for card in player_cards]

        improving_cards = 0
        for card in deck:
            makes_pair = card.rank in all_ranks
            beats_known = any(card.score() < s for s in all_scores)
            if makes_pair or beats_known:
                improving_cards += 1

        prob = improving_cards / len(deck)
        results.append(f'{round(prob * 100, 1)}%')

    return results





def get_probabilities(game):
    """Return a dict of interesting probabilities/statistics for the current game state."""
    return {
        'deck_counts': get_deck_counts(game),
        'prob_draw_lower_than_min_faceup': prob_draw_lower_than_min_faceup(game),
        'prob_draw_pair': prob_draw_pair(game),
        'prob_improve_hand': prob_improve_hand(game),
    }



In [ ]:
import requests
import json
import os
from dotenv import load_dotenv

def load_existing_gifs(filename='golf_celebration_gifs.json'):
    """Load existing GIFs from JSON file and return set of IDs"""
    if not os.path.exists(filename):
        return set(), {'data': []}

    try:
        with open(filename, 'r') as f:
            existing_data = json.load(f)

        # Extract existing GIF IDs
        existing_ids = {gif['id'] for gif in existing_data.get('data', [])}
        return existing_ids, existing_data
    except (json.JSONDecodeError, KeyError):
        print("Error reading existing file, starting fresh")
        return set(), {'data': []}

def get_giphy_data(search_term, limit=50, offset=0):
    """Fetch GIF data from Giphy API"""
    load_dotenv()
    api_key = os.getenv('GIPHY_API_KEY')

    if not api_key:
        raise ValueError("GIPHY_API_KEY not found in environment variables")

    url = f"https://api.giphy.com/v1/gifs/search"

    params = {
        'api_key': api_key,
        'q': search_term,
        'limit': limit,
        'offset': offset
    }

    response = requests.get(url, params=params)
    response.raise_for_status()  # Raise an error for bad status codes

    return response.json()

def filter_gif_data(gif):
    """Extract only the keys we want from each GIF"""
    return {
        'id': gif.get('id'),
        'title': gif.get('title'),
        'url': gif.get('url'),
        'rating': gif.get('rating'),
        'images': {
            'downsized_large': gif.get('images', {}).get('downsized_large', {}),
            'downsized_medium': gif.get('images', {}).get('downsized_medium', {}),
            'downsized_small': gif.get('images', {}).get('downsized_small', {})
        }
    }

def update_gif_collection(search_term, filename='golf_celebration_gifs.json', batch_size=50, max_requests=5):
    """Update GIF collection with only new GIFs"""
    filename = filename

    # Load existing GIFs
    existing_ids, existing_data = load_existing_gifs(filename)
    print(f"Found {len(existing_ids)} existing GIFs")

    all_gifs = existing_data.get('data', [])
    new_gifs_count = 0
    total_processed = 0

    # Fetch new GIFs in batches
    for batch in range(max_requests):
        offset = batch * batch_size
        print(f"Fetching batch {batch + 1} (offset: {offset})...")

        try:
            new_data = get_giphy_data(search_term, limit=batch_size, offset=offset)
            new_gifs = new_data.get('data', [])

            if not new_gifs:
                print("No more GIFs found")
                break

            # Filter out existing GIFs and clean the data
            truly_new_gifs = [
                filter_gif_data(gif)
                for gif in new_gifs
                if gif['id'] not in existing_ids
            ]

            if truly_new_gifs:
                all_gifs.extend(truly_new_gifs)
                # Add new IDs to existing set
                existing_ids.update(gif['id'] for gif in truly_new_gifs)
                new_gifs_count += len(truly_new_gifs)
                print(f"Added {len(truly_new_gifs)} new GIFs from this batch")
            else:
                print("No new GIFs in this batch")

            total_processed += len(new_gifs)

        except requests.RequestException as e:
            print(f"Error fetching data: {e}")
            break

    # Save updated collection
    updated_data = {
        'data': all_gifs,
        'pagination': {
            'total_count': len(all_gifs),
            'count': len(all_gifs),
            'offset': 0
        },
        'meta': {
            'status': 200,
            'msg': 'OK',
            'response_id': 'updated_collection'
        }
    }

    with open(filename, 'w') as f:
        json.dump(updated_data, f, indent=2)

    print(f"\nSummary:")
    print(f"- Total GIFs processed: {total_processed}")
    print(f"- New GIFs added: {new_gifs_count}")
    print(f"- Total GIFs in collection: {len(all_gifs)}")
    print(f"- File saved: {filename}")

    return updated_data

# Usage
if __name__ == "__main__":
    # Update collection with new GIFs
    gif_data = update_gif_collection('golf-celebration', filename='golf_celebration_gifs.json', batch_size=50, max_requests=10)